In [ ]:
#df1 = df.round()
#df1 = df1.drop_duplicates(subset=['min_distance1', 'min_distance2'])
#print(df1.shape)
#print(df1)

In [2]:
import numpy as np
import plotly.graph_objects as go
import itertools
import pandas as pd
from scipy.spatial import KDTree
import matplotlib.pyplot as plt

In [ ]:

# Function to generate a cylinder with variable radius

def generate_slipstream(rad, z_sections, radius_funcs, resolution_theta=50, resolution_z=50):
    z_min, z_max = min(z_sections), max(z_sections)
    z_steps = np.linspace(z_min, z_max, resolution_z)
    theta = np.linspace(0, 2 * np.pi, resolution_theta)

    radius = np.zeros_like(z_steps)
    for i in range(len(z_sections) - 1):
        mask = (z_steps >= z_sections[i]) & (z_steps <= z_sections[i + 1])
        radius[mask] = rad * radius_funcs[i](z_steps[mask])

    # Avoid creating full grids
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    x = radius[:, None] * cos_theta
    y = radius[:, None] * sin_theta
    z = (z_steps * rad * 2)[:, None] + np.zeros_like(theta)

    # Combine points efficiently
    points = np.column_stack((x.ravel(), y.ravel(), z.ravel()))
    return points


# Define radius functions
#def radius_above(z): return 1 + z *0.25
#def radius_contract(z): return 1 + 0*z
#def radius_nf(z): return 1 - (z+0.9)*0.4
#def radius_ff(z): return 1.3 - (z+1.6)*0.3
#radius_funcs = [radius_ff, radius_nf, radius_contract, radius_above]

def radius_above(z): return 1 +z*0
def radius_contract(z): return 1 + z*0
radius_funcs = [radius_above, radius_contract]

# Function to generate surface grids for plotting
def generate_surface_grids(points, resolution_theta, resolution_z):
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    x_grid = x.reshape(resolution_z, resolution_theta)
    y_grid = y.reshape(resolution_z, resolution_theta)
    z_grid = z.reshape(resolution_z, resolution_theta)
    return x_grid, y_grid, z_grid


# Function to create rotation matrices for given angles
def create_transformation_matrix(a1, a2):
    TI1_1 = np.array([
        [1, 0, 0, 0],
        [0, np.cos(a1), -np.sin(a1), 0],
        [0, np.sin(a1), np.cos(a1), 0],
        [0, 0, 0, 1]
    ])
    T1_2 = np.array([
        [1, 0, 0, armlength],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])

    TI2_1 = np.array([
        [np.cos(-np.pi / 3), -np.sin(-np.pi / 3), 0, 0],
        [np.sin(-np.pi / 3), np.cos(-np.pi / 3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T2_2 = np.array([
        [1, 0, 0, armlength],
        [0, np.cos(a2), -np.sin(a2), 0],
        [0, np.sin(a2), np.cos(a2), 0],
        [0, 0, 0, 1]
    ])
    TI2_3 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])
    TI1 = np.matmul(TI1_1,T1_2)
    TI2 = np.matmul(np.matmul(TI2_1, T2_2), TI2_3)
    return TI1, TI2

def invert_transformation(matrix):
    """
    Computes the inverse of a 4x4 homogeneous transformation matrix.
    
    Args:
        T (np.ndarray): A 4x4 homogeneous transformation matrix.
    
    Returns:
        np.ndarray: The inverse of the input transformation matrix.
    """
    # Extract rotation matrix (3x3) and translation vector (3x1)
    R = matrix[:3, :3]
    t = matrix[:3, 3]
    
    # Inverse of rotation is just the transpose of R
    R_inv = R.T
    
    # Inverse of translation is -R_inv * t
    t_inv = -np.dot(R_inv, t)
    
    # Construct the inverse transformation matrix
    matrix_inv = np.eye(4)
    matrix_inv[:3, :3] = R_inv
    matrix_inv[:3, 3] = t_inv
    
    return matrix_inv


# Transformation matrix
def apply_transform(matrix, points):
    num_points = points.shape[0]
    homogeneous_points = np.c_[points, np.ones(num_points)]  # Inline addition of the homogeneous coordinate
    return np.dot(homogeneous_points, matrix.T)[:, :3]


#def find_intersection_points(cylinder1_kdtree, cylinder1_points, cylinder2_points, ti1, ti2, t1i, t2i, threshold=0.1):
    """
    Finds intersection points between two cylinders, transforms them back, and returns the closest Z points.
    
    Args:
        cylinder1_kdtree: KDTree for Cylinder 1.
        cylinder1_points: Points of Cylinder 1 in its local frame.
        cylinder2_points: Points of Cylinder 2 in its local frame.
        ti1_inv: Inverse transformation matrix for Cylinder 1.
        ti2_inv: Inverse transformation matrix for Cylinder 2.
        threshold: Distance threshold to consider points intersecting.
    
    Returns:
        - Transformed intersection points back into the inertial frame.
        - Closest point with minimum +Z and -Z for both cylinders.
        - Z values and coordinates in the inertial frame.
    """

    # Step 1: Query KDTree for intersections within a threshold
    distances, indices = cylinder1_kdtree.query(cylinder2_points, distance_upper_bound=threshold)
    valid_mask = distances < threshold
    intersection_points_cyl2 = cylinder2_points[valid_mask]

    # Step 2: Transform intersection points back to Cylinder 1 and 2 local coordinates
    intersection_points_cyl1 = cylinder1_points[indices[valid_mask]]
    if len(intersection_points_cyl1) == 0:
        return [], None, None, None, None, None, None, None, None

    # Transform using inverse matrices to local cylinder coordinates
    intersection_points_cyl1_transformed = apply_transform(t1i,intersection_points_cyl1)
    intersection_points_cyl2_transformed = apply_transform(t2i,intersection_points_cyl2)
    # Step 3: Find min +Z and -Z values and corresponding points
    #print(intersection_points_cyl1)
    #print(intersection_points_cyl1_transformed)

    # Initialize the result variables in case no valid points are found
    min_positive_z_cyl1 = min_negative_z_cyl1 = None
    min_positive_z_cyl1_idx = min_negative_z_cyl1_idx = None
    min_positive_z_cyl2 = min_negative_z_cyl2 = None
    min_positive_z_cyl2_idx = min_negative_z_cyl2_idx = None
    min_distance_pos_1 = min_distance_neg_1 = min_distance_pos_2 = min_distance_neg_2 = None

    # Step 1: Find the index and points with non-negative and non-positive Z-values for Cylinder 1
    if np.any(intersection_points_cyl1_transformed[:, 2] >= 0):
        # Filter non-negative Z-values (including zero)
        non_negative_z_values_cyl1 = intersection_points_cyl1_transformed[:, 2][intersection_points_cyl1_transformed[:, 2] >= 0]
        min_positive_z_cyl1_idx_filtered = np.argmin(np.abs(non_negative_z_values_cyl1))
        # Map back to original index
        min_positive_z_cyl1_idx = np.where(intersection_points_cyl1_transformed[:, 2] >= 0)[0][min_positive_z_cyl1_idx_filtered]
        min_positive_z_cyl1 = intersection_points_cyl1_transformed[min_positive_z_cyl1_idx].reshape(1, 3)
        #print(min_positive_z_cyl1)
        min_distance_pos_1 = min_positive_z_cyl1[0, 2]

    if np.any(intersection_points_cyl1_transformed[:, 2] <= 0):
        # Filter non-positive Z-values (including zero)
        non_positive_z_values_cyl1 = intersection_points_cyl1_transformed[:, 2][intersection_points_cyl1_transformed[:, 2] <= 0]
        min_negative_z_cyl1_idx_filtered = np.argmin(np.abs(non_positive_z_values_cyl1))
        # Map back to original index
        min_negative_z_cyl1_idx = np.where(intersection_points_cyl1_transformed[:, 2] <= 0)[0][min_negative_z_cyl1_idx_filtered]
        min_negative_z_cyl1 = intersection_points_cyl1_transformed[min_negative_z_cyl1_idx].reshape(1, 3)
        #print(min_negative_z_cyl1)
        min_distance_neg_1 = min_negative_z_cyl1[0, 2]

    # Step 2: Find the index and points with non-negative and non-positive Z-values for Cylinder 2
    if np.any(intersection_points_cyl2_transformed[:, 2] >= 0):
        # Filter non-negative Z-values (including zero)
        non_negative_z_values_cyl2 = intersection_points_cyl2_transformed[:, 2][intersection_points_cyl2_transformed[:, 2] >= 0]
        min_positive_z_cyl2_idx_filtered = np.argmin(np.abs(non_negative_z_values_cyl2))
        # Map back to original index
        min_positive_z_cyl2_idx = np.where(intersection_points_cyl2_transformed[:, 2] >= 0)[0][min_positive_z_cyl2_idx_filtered]
        min_positive_z_cyl2 = intersection_points_cyl2_transformed[min_positive_z_cyl2_idx].reshape(1, 3)
        #print(min_positive_z_cyl2)
        min_distance_pos_2 = min_positive_z_cyl2[0, 2]

    if np.any(intersection_points_cyl2_transformed[:, 2] <= 0):
        # Filter non-positive Z-values (including zero)
        non_positive_z_values_cyl2 = intersection_points_cyl2_transformed[:, 2][intersection_points_cyl2_transformed[:, 2] <= 0]
        min_negative_z_cyl2_idx_filtered = np.argmin(np.abs(non_positive_z_values_cyl2))
        # Map back to original index
        min_negative_z_cyl2_idx = np.where(intersection_points_cyl2_transformed[:, 2] <= 0)[0][min_negative_z_cyl2_idx_filtered]
        min_negative_z_cyl2 = intersection_points_cyl2_transformed[min_negative_z_cyl2_idx].reshape(1, 3)
        #print(min_negative_z_cyl2)
        min_distance_neg_2 = min_negative_z_cyl2[0, 2]


    # Step 3: Transform the points back to the inertial frame, if they were found
    min_positive_z_cyl1_inertial = apply_transform(ti1, min_positive_z_cyl1) if min_positive_z_cyl1 is not None else None
    #print(min_negative_z_cyl1.shape)
    #print(min_negative_z_cyl1)
    min_negative_z_cyl1_inertial = apply_transform(ti1, min_negative_z_cyl1) if min_negative_z_cyl1 is not None else None
    min_positive_z_cyl2_inertial = apply_transform(ti2, min_positive_z_cyl2) if min_positive_z_cyl2 is not None else None
    min_negative_z_cyl2_inertial = apply_transform(ti2, min_negative_z_cyl2) if min_negative_z_cyl2 is not None else None

    return (
        intersection_points_cyl1,
        min_distance_pos_1,
        min_distance_neg_1,
        min_distance_pos_2,
        min_distance_neg_2,
        min_positive_z_cyl1_inertial,
        min_negative_z_cyl1_inertial,
        min_positive_z_cyl2_inertial,
        min_negative_z_cyl2_inertial
    )

def find_intersection_points(cylinder1_kdtree, cylinder1_points, cylinder2_points, ti1, ti2, t1i, t2i, threshold=0.1):
    """
    Finds intersection points between two cylinders, transforms them back, and returns the closest Z points.
    
    Args:
        cylinder1_kdtree: KDTree for Cylinder 1.
        cylinder1_points: Points of Cylinder 1 in its local frame.
        cylinder2_points: Points of Cylinder 2 in its local frame.
        ti1_inv: Inverse transformation matrix for Cylinder 1.
        ti2_inv: Inverse transformation matrix for Cylinder 2.
        threshold: Distance threshold to consider points intersecting.
    
    Returns:
        - Transformed intersection points back into the inertial frame.
        - Closest point with minimum +Z and -Z for both cylinders.
        - Z values and coordinates in the inertial frame.
    """

    # Step 1: Query KDTree for intersections within a threshold
    distances, indices = cylinder1_kdtree.query(cylinder2_points, distance_upper_bound=threshold)
    valid_mask = distances < threshold
    intersection_points_cyl2 = cylinder2_points[valid_mask]

    # Step 2: Transform intersection points back to Cylinder 1 and 2 local coordinates
    intersection_points_cyl1 = cylinder1_points[indices[valid_mask]]
    if len(intersection_points_cyl1) == 0:
        return [], None, None, None, None, None, None, None, None

    # Transform using inverse matrices to local cylinder coordinates
    intersection_points_cyl1_transformed = apply_transform(t1i,intersection_points_cyl1)
    intersection_points_cyl2_transformed = apply_transform(t2i,intersection_points_cyl2)
    # Step 3: Find min +Z and -Z values and corresponding points
    #print(intersection_points_cyl1)
    #print(intersection_points_cyl1_transformed)

    # Initialize the result variables in case no valid points are found
    min_cyl1 = max_cyl1 = None
    min_cyl1_idx = max_cyl1_idx = None
    min_cyl2 = max_cyl2 = None
    min_cyl2_idx = max_cyl2_idx = None
    min_distance_cyl_1 = max_distance_cyl_1 = min_distance_cyl_2 = max_distance_cyl_2 = None


    if np.any(intersection_points_cyl1_transformed):
        min_cyl1_idx = np.argmin(intersection_points_cyl1_transformed[:, 2])
        min_cyl1 = intersection_points_cyl1_transformed[min_cyl1_idx].reshape(1, 3)
        min_distance_cyl_1 = min_cyl1[0, 2]

        max_cyl1_idx = np.argmax(intersection_points_cyl1_transformed[:, 2])
        max_cyl1 = intersection_points_cyl1_transformed[max_cyl1_idx].reshape(1, 3)
        max_distance_cyl_1 = max_cyl1[0, 2]


    if np.any(intersection_points_cyl2_transformed):
        min_cyl2_idx = np.argmin(intersection_points_cyl2_transformed[:, 2])
        min_cyl2 = intersection_points_cyl2_transformed[min_cyl2_idx].reshape(1, 3)
        min_distance_cyl_2 = min_cyl2[0, 2]

        max_cyl2_idx = np.argmax(intersection_points_cyl2_transformed[:, 2])
        max_cyl2 = intersection_points_cyl2_transformed[max_cyl2_idx].reshape(1, 3)
        max_distance_cyl_2 = max_cyl2[0, 2]




    # Step 3: Transform the points back to the inertial frame, if they were found
    min_cyl1_inertial = apply_transform(ti1, min_cyl1) if min_cyl1 is not None else None
    #print(min_negative_z_cyl1.shape)
    #print(min_negative_z_cyl1)
    max_cyl1_inertial = apply_transform(ti1, max_cyl1) if max_cyl1 is not None else None
    min_cyl2_inertial = apply_transform(ti2, min_cyl2) if min_cyl2 is not None else None
    max_cyl2_inertial = apply_transform(ti2, max_cyl2) if max_cyl2 is not None else None

    return (
        intersection_points_cyl1,
        min_distance_cyl_1,
        max_distance_cyl_1,
        min_distance_cyl_2,
        max_distance_cyl_2,
        min_cyl1_inertial,
        max_cyl1_inertial,
        min_cyl2_inertial,
        max_cyl2_inertial
    )

def downsample_grid(grid, factor):
    return grid[::factor, ::factor]


# Function to plot the cylinders, intersection points, and closest points
def plot_cylinders(cylinder1_surface, cylinder2_surface, intersection_points_cyl1, min_distance_cyl_1, max_distance_cyl_1,
                    min_distance_cyl_2, max_distance_cyl_2, 
                    min_cyl1_inertial, max_cyl1_inertial, min_cyl2_inertial, max_cyl2_inertial,
                    center1, center2, a1, a2):
    fig = go.Figure()

    # Add Cylinder 1 as a surface
    fig.add_trace(go.Surface(
        x=cylinder1_surface[0], y=cylinder1_surface[1], z=cylinder1_surface[2],
        colorscale='Blues',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 1 (a1={a1:.2f}, a2={a2:.2f})'
    ))

    # Add Cylinder 2 as a surface
    fig.add_trace(go.Surface(
        x=cylinder2_surface[0], y=cylinder2_surface[1], z=cylinder2_surface[2],
        colorscale='Reds',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 2 (a1={a1:.2f}, a2={a2:.2f})'
    ))

    # Add vector from origin to Cylinder 1 center
    fig.add_trace(go.Scatter3d(
        x=[0, 271.5 + 124], y=[0, 0], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Vector to Arm 1'
    ))
    fig.add_trace(go.Scatter3d(
        x=[271.5 + 124, center1[0,0]], y=[0, center1[0,1]], z=[0, center1[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Vector to Arm 1'
    ))

    # Add vector from origin to Cylinder 2 center
    fig.add_trace(go.Scatter3d(
        x=[0, (271.5 + 124) * np.cos(np.pi / 3)], y=[0, (271.5 + 124) * np.sin(np.pi / 3)], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Vector to Arm 2'
    ))
    fig.add_trace(go.Scatter3d(
        x=[(271.5 + 124) * np.cos(np.pi / 3), center2[0,0]], y=[(271.5 + 124) * np.sin(np.pi / 3), center2[0,1] ], z=[0, center2[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Vector to Arm 2'
    ))

    # Add Intersection Points
    if len(intersection_points_cyl1) > 0:
        fig.add_trace(go.Scatter3d(
            x=intersection_points_cyl1[:, 0], y=intersection_points_cyl1[:, 1], z=intersection_points_cyl1[:, 2],
            mode='markers', marker=dict(size=4, color='green', opacity=0.8), name=f'Intersections'
        ))


    # Add Min Positive and Negative Z Points for Cylinder 1
    if min_cyl1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl1_inertial[0,0]], y=[min_cyl1_inertial[0,1]], z=[min_cyl1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder 1, Distance={min_distance_cyl_1:.2f}'
        ))
    if max_cyl1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl1_inertial[0,0]], y=[max_cyl1_inertial[0,1]], z=[max_cyl1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder 1, Distance={max_distance_cyl_1:.2f}'
        ))

    if min_cyl2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl2_inertial[0,0]], y=[min_cyl2_inertial[0,1]], z=[min_cyl2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder 2, Distance={min_distance_cyl_2:.2f}'
        ))
    if max_cyl2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl2_inertial[0,0]], y=[max_cyl2_inertial[0,1]], z=[max_cyl2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder 2, Distance={max_distance_cyl_2:.2f}'
        ))

    # Customize layout
    fig.update_layout(
        title=f"Cylinder Intersection and Range of Intersection (a1={np.degrees(a1):.1f}°, a2={np.degrees(a2):.1f}°)",
        scene=dict(
            aspectmode='cube',
            xaxis=dict(title="X-axis", range=[-1000, 1000]),
            yaxis=dict(title="Y-axis", range=[-1000, 1000]),
            zaxis=dict(title="Z-axis", range=[-1000, 1000]),
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        showlegend=True
    )

    # Show plot
    fig.show()


# Define parameters
threshold = 1
radius_prop = 304.8/2
armlength = 271.5 + 124
z_offset = -52.7
#z_offset = 0

# Normalised with prop radius
z_sections = [0.5, 0, -0.9, -1.6, -4.5]  # Positive = suction
z_sections = [1, 0, -1]                  # Positive = suction
z_sections = sorted(z_sections, reverse=False)

resolution_theta = int(np.round(2*np.pi*radius_prop/threshold/1.4))
resolution_z = int(np.round(2*radius_prop*(max(z_sections) - min(z_sections))/threshold/1.4))
#print(resolution_theta)
#print(resolution_z)


data = []

# Define angles to iterate over
A1 = np.linspace(0, 2 * np.pi, 6, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 6, endpoint=False)
A1 = np.linspace(0, 2 * np.pi, 36, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 36, endpoint=False)
A1 = np.linspace(0, 2 * np.pi, 144, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 72, endpoint=False)
#A2 = np.linspace(7*np.pi/6, 11 * np.pi/6, 49) #left direct impingement 
A2 = np.linspace(np.pi/6, 5 * np.pi/6, 49) #right direct impingement 
#A1 = [np.radians(60)]
angList = list(itertools.product(A1, A2))
#angList = [(np.radians(210), np.radians(210)), (np.radians(210), np.radians(240)), (np.radians(210), np.radians(270)), (np.radians(330), np.radians(300))]
#angList = [(np.radians(210), np.radians(210)), (np.radians(210), np.radians(240)), (np.radians(210), np.radians(270))]
#print(f"Angle from `angList` using linspace: {angList[92]}")
#print(f"Manually input angle (210, 240): {(np.radians(210), np.radians(240))}")
#
#print("Difference in angles:")
#print(np.array(angList[92]) - np.array((np.radians(210), np.radians(240))))
#print("Are they close?:", np.isclose(angList[92], (np.radians(210), np.radians(240))))
#print(angList)

angList = [(np.round(a[0], 8), np.round(a[1], 8)) for a in angList]
# Initialize variables to track the previous state of a1 and cached data
previous_a1 = None
cylinder1_points = None
cylinder1_kdtree = None
cylinder1_surface = None
cylinder2_surface = None

# Iterate through each combination of angles
for a1, a2 in angList:
    # Check if a1 has changed
    if a1 != previous_a1:
        # Update cylinder1 and its associated data only when a1 changes
        TI1, _ = create_transformation_matrix(a1, 0)  # Only need TI1 for cylinder1
        T1I = invert_transformation(TI1)
        origin_point = np.zeros((1, 3))
        center1 = apply_transform(TI1, origin_point)
        
        # Generate and transform Cylinder 1
        cylinder1_points = generate_slipstream(
            radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
        )
        cylinder1_points = apply_transform(TI1, cylinder1_points)
        
        # Build KDTree for Cylinder 1
        cylinder1_kdtree = KDTree(cylinder1_points)
        
        # Generate surface grids for Cylinder 1 (downsample for plotting)
        #cylinder1_surface = generate_surface_grids(cylinder1_points, resolution_theta, resolution_z)
        #cylinder1_surface = tuple(downsample_grid(g, factor=5) for g in cylinder1_surface)

        # Update the cached `previous_a1`
        previous_a1 = a1

    # Process Cylinder 2 for the current a2
    _, TI2 = create_transformation_matrix(a1, a2)
    T2I = invert_transformation(TI2)
    origin_point = np.zeros((1, 3))
    center2 = apply_transform(TI2, origin_point)
    
    # Generate and transform Cylinder 2
    cylinder2_points = generate_slipstream(
        radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
    )
    #print(cylinder2_points.shape)
    cylinder2_points = apply_transform(TI2, cylinder2_points)
    #print(cylinder2_points.shape)
    # Find intersection points using precomputed KDTree for Cylinder 1
    (intersection_points_cyl1,
    min_distance_cyl_1,
    max_distance_cyl_1,
    min_distance_cyl_2,
    max_distance_cyl_2,
    min_cyl1_inertial,
    max_cyl1_inertial,
    min_cyl2_inertial,
    max_cyl2_inertial) = find_intersection_points(
    cylinder1_kdtree, cylinder1_points, cylinder2_points, TI1, TI2, T1I, T2I, threshold)

    # Generate surface grids for Cylinder 2 (downsample for plotting)
    #cylinder2_surface = generate_surface_grids(cylinder2_points, resolution_theta, resolution_z)
    #cylinder2_surface = tuple(downsample_grid(g, factor=5) for g in cylinder2_surface)

    # Plot cylinders and intersection points
    #plot_cylinders(cylinder1_surface, cylinder2_surface, intersection_points_cyl1, min_distance_cyl_1, max_distance_cyl_1,
    #                min_distance_cyl_2, max_distance_cyl_2, 
    #                min_cyl1_inertial, max_cyl1_inertial, min_cyl2_inertial, max_cyl2_inertial,
    #                center1, center2, a1, a2)

    # Store results in the dataframe
    data.append([np.degrees(a1), np.degrees(a2), min_distance_cyl_1, max_distance_cyl_1,
    min_distance_cyl_2, max_distance_cyl_2])
            

# Create the DataFrame from collected data
df = pd.DataFrame(data, columns=['a1', 'a2', 'min_distance_cyl_1', 'max_distance_cyl_1',
    'min_distance_cyl_2', 'max_distance_cyl_2'])
print(df.shape)
#df.replace([np.inf, -np.inf], np.nan, inplace=True)
#df = df.dropna()
#print(df.shape)
print(df.round(1).to_string())



In [50]:
# Save the DataFrame as a CSV file with semicolon delimiters
csv_path = "./raw_right.csv"  # File will be saved in the same folder as the notebook
df.round(1).to_csv(csv_path, sep=';', index=False, header=False)

#df = pd.read_csv('./raw_1_2.csv', sep=';', names=['a1', 'a2', 'min_distance_cyl_1', 'max_distance_cyl_1',
 #   'min_distance_cyl_2', 'max_distance_cyl_2'], header=None)

In [17]:
df = pd.read_csv('./raw_left.csv', sep=';', names=['a1', 'a2', 'min_distance_cyl_1', 'max_distance_cyl_1',
    'min_distance_cyl_2', 'max_distance_cyl_2'], header=None)

In [ ]:
print(df.to_string())

In [ ]:
df_cleaned = df.dropna(subset='min_distance_cyl_2') # ensures the slipstreams hit eachother?
df_cleaned = df_cleaned[df_cleaned['min_distance_cyl_2'] <= 0] # only the blowing side of the blower is considered
df_cleaned = df_cleaned[(df_cleaned['min_distance_cyl_1'] <= 0) & (df_cleaned['max_distance_cyl_1'] >= 0)]
#df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned = df_cleaned.round(1)
print(df_cleaned.shape)
print(df_cleaned.to_string())

In [ ]:
df_sorted_sym = df_cleaned.sort_values(['min_distance_cyl_1', 'max_distance_cyl_1','min_distance_cyl_2', 'max_distance_cyl_2'])
df_string_sym = df_sorted_sym.to_string()
print(df_string_sym) 
#both sides add up to 180 deg

In [ ]:
# Ensure df_cleaned is not modified
df_copy = df_cleaned.copy()

# Identify duplicates based on all columns except 'a1' and 'a2'
duplicates = df_copy.duplicated(subset=df_copy.columns.difference(['a1', 'a2']), keep='first')

# Extract remaining points and dropped points without modifying the original DataFrame
df_remaining = df_copy[~duplicates].copy()
df_dropped = df_copy[duplicates].copy()

# Calculate axis limits for consistency
a1_min = df_copy['a1'].min()
a1_max = df_copy['a1'].max()
a2_min = df_copy['a2'].min()
a2_max = df_copy['a2'].max()

# Plot Remaining Points in a Separate Figure
plt.figure(figsize=(8, 6))
plt.scatter(df_remaining['a1'], df_remaining['a2'], color='blue', label='Remaining Points')
plt.title("Remaining Points")
plt.xlabel('a1')
plt.ylabel('a2')
plt.xlim(a1_min, a1_max)  # Set consistent x-axis limits
plt.ylim(a2_min, a2_max)  # Set consistent y-axis limits
plt.legend()
plt.grid(True)
plt.show()

# Plot Dropped Points in a Separate Figure
plt.figure(figsize=(8, 6))
plt.scatter(df_dropped['a1'], df_dropped['a2'], color='red', label='Dropped Points')
plt.title("Dropped Points")
plt.xlabel('a1')
plt.ylabel('a2')
plt.xlim(a1_min, a1_max)  # Set consistent x-axis limits
plt.ylim(a2_min, a2_max)  # Set consistent y-axis limits
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
#df_cleaned = df_cleaned.sort_values('a1', ascending=False)

df_cleaned = df_cleaned.drop_duplicates(subset=df_cleaned.columns.difference(['a1', 'a2']), keep='first')
print(df_cleaned.shape)
print(df_cleaned.to_string())

In [ ]:
df_sorted_sym = df_cleaned.sort_values(['min_distance_cyl_1', 'max_distance_cyl_1','min_distance_cyl_2', 'max_distance_cyl_2'])
df_string_sym = df_sorted_sym.to_string()
print(df_string_sym) 

In [ ]:
import numpy as np
import plotly.graph_objects as go
import itertools
import pandas as pd
from scipy.spatial import KDTree

# Function to generate a cylinder with variable radius

def generate_slipstream(rad, z_sections, radius_funcs, resolution_theta=50, resolution_z=50):
    z_min, z_max = min(z_sections), max(z_sections)
    z_steps = np.linspace(z_min, z_max, resolution_z)
    theta = np.linspace(0, 2 * np.pi, resolution_theta)

    radius = np.zeros_like(z_steps)
    for i in range(len(z_sections) - 1):
        mask = (z_steps >= z_sections[i]) & (z_steps <= z_sections[i + 1])
        radius[mask] = rad * radius_funcs[i](z_steps[mask])

    # Avoid creating full grids
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    x = radius[:, None] * cos_theta
    y = radius[:, None] * sin_theta
    z = (z_steps * rad * 2)[:, None] + np.zeros_like(theta)

    # Combine points efficiently
    points = np.column_stack((x.ravel(), y.ravel(), z.ravel()))
    return points


# Define radius functions
#def radius_above(z): return 1 + z *0.25
#def radius_contract(z): return 1 + 0*z
#def radius_nf(z): return 1 - (z+0.9)*0.4
#def radius_ff(z): return 1.3 - (z+1.6)*0.3
#radius_funcs = [radius_ff, radius_nf, radius_contract, radius_above]

def radius_above(z): return 1 +z*0
def radius_contract(z): return 1 + z*0
radius_funcs = [radius_above, radius_contract]

# Function to generate surface grids for plotting
def generate_surface_grids(points, resolution_theta, resolution_z):
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    x_grid = x.reshape(resolution_z, resolution_theta)
    y_grid = y.reshape(resolution_z, resolution_theta)
    z_grid = z.reshape(resolution_z, resolution_theta)
    return x_grid, y_grid, z_grid


# Function to create rotation matrices for given angles
def create_transformation_matrix(a1, a2):
    TI1_1 = np.array([
        [1, 0, 0, 0],
        [0, np.cos(a1), -np.sin(a1), 0],
        [0, np.sin(a1), np.cos(a1), 0],
        [0, 0, 0, 1]
    ])
    T1_2 = np.array([
        [1, 0, 0, armlength],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])

    TI2_1 = np.array([
        [np.cos(np.pi / 3), -np.sin(np.pi / 3), 0, 0],
        [np.sin(np.pi / 3), np.cos(np.pi / 3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T2_2 = np.array([
        [1, 0, 0, armlength],
        [0, np.cos(a2), -np.sin(a2), 0],
        [0, np.sin(a2), np.cos(a2), 0],
        [0, 0, 0, 1]
    ])
    TI2_3 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])
    TI1 = np.matmul(TI1_1,T1_2)
    TI2 = np.matmul(np.matmul(TI2_1, T2_2), TI2_3)
    return TI1, TI2

def invert_transformation(matrix):
    """
    Computes the inverse of a 4x4 homogeneous transformation matrix.
    
    Args:
        T (np.ndarray): A 4x4 homogeneous transformation matrix.
    
    Returns:
        np.ndarray: The inverse of the input transformation matrix.
    """
    # Extract rotation matrix (3x3) and translation vector (3x1)
    R = matrix[:3, :3]
    t = matrix[:3, 3]
    
    # Inverse of rotation is just the transpose of R
    R_inv = R.T
    
    # Inverse of translation is -R_inv * t
    t_inv = -np.dot(R_inv, t)
    
    # Construct the inverse transformation matrix
    matrix_inv = np.eye(4)
    matrix_inv[:3, :3] = R_inv
    matrix_inv[:3, 3] = t_inv
    
    return matrix_inv


# Transformation matrix
def apply_transform(matrix, points):
    num_points = points.shape[0]
    homogeneous_points = np.c_[points, np.ones(num_points)]  # Inline addition of the homogeneous coordinate
    return np.dot(homogeneous_points, matrix.T)[:, :3]


#def find_intersection_points(cylinder1_kdtree, cylinder1_points, cylinder2_points, ti1, ti2, t1i, t2i, threshold=0.1):
    """
    Finds intersection points between two cylinders, transforms them back, and returns the closest Z points.
    
    Args:
        cylinder1_kdtree: KDTree for Cylinder 1.
        cylinder1_points: Points of Cylinder 1 in its local frame.
        cylinder2_points: Points of Cylinder 2 in its local frame.
        ti1_inv: Inverse transformation matrix for Cylinder 1.
        ti2_inv: Inverse transformation matrix for Cylinder 2.
        threshold: Distance threshold to consider points intersecting.
    
    Returns:
        - Transformed intersection points back into the inertial frame.
        - Closest point with minimum +Z and -Z for both cylinders.
        - Z values and coordinates in the inertial frame.
    """

    # Step 1: Query KDTree for intersections within a threshold
    distances, indices = cylinder1_kdtree.query(cylinder2_points, distance_upper_bound=threshold)
    valid_mask = distances < threshold
    intersection_points_cyl2 = cylinder2_points[valid_mask]

    # Step 2: Transform intersection points back to Cylinder 1 and 2 local coordinates
    intersection_points_cyl1 = cylinder1_points[indices[valid_mask]]
    if len(intersection_points_cyl1) == 0:
        return [], None, None, None, None, None, None, None, None

    # Transform using inverse matrices to local cylinder coordinates
    intersection_points_cyl1_transformed = apply_transform(t1i,intersection_points_cyl1)
    intersection_points_cyl2_transformed = apply_transform(t2i,intersection_points_cyl2)
    # Step 3: Find min +Z and -Z values and corresponding points
    #print(intersection_points_cyl1)
    #print(intersection_points_cyl1_transformed)

    # Initialize the result variables in case no valid points are found
    min_positive_z_cyl1 = min_negative_z_cyl1 = None
    min_positive_z_cyl1_idx = min_negative_z_cyl1_idx = None
    min_positive_z_cyl2 = min_negative_z_cyl2 = None
    min_positive_z_cyl2_idx = min_negative_z_cyl2_idx = None
    min_distance_pos_1 = min_distance_neg_1 = min_distance_pos_2 = min_distance_neg_2 = None

    # Step 1: Find the index and points with non-negative and non-positive Z-values for Cylinder 1
    if np.any(intersection_points_cyl1_transformed[:, 2] >= 0):
        # Filter non-negative Z-values (including zero)
        non_negative_z_values_cyl1 = intersection_points_cyl1_transformed[:, 2][intersection_points_cyl1_transformed[:, 2] >= 0]
        min_positive_z_cyl1_idx_filtered = np.argmin(np.abs(non_negative_z_values_cyl1))
        # Map back to original index
        min_positive_z_cyl1_idx = np.where(intersection_points_cyl1_transformed[:, 2] >= 0)[0][min_positive_z_cyl1_idx_filtered]
        min_positive_z_cyl1 = intersection_points_cyl1_transformed[min_positive_z_cyl1_idx].reshape(1, 3)
        #print(min_positive_z_cyl1)
        min_distance_pos_1 = min_positive_z_cyl1[0, 2]

    if np.any(intersection_points_cyl1_transformed[:, 2] <= 0):
        # Filter non-positive Z-values (including zero)
        non_positive_z_values_cyl1 = intersection_points_cyl1_transformed[:, 2][intersection_points_cyl1_transformed[:, 2] <= 0]
        min_negative_z_cyl1_idx_filtered = np.argmin(np.abs(non_positive_z_values_cyl1))
        # Map back to original index
        min_negative_z_cyl1_idx = np.where(intersection_points_cyl1_transformed[:, 2] <= 0)[0][min_negative_z_cyl1_idx_filtered]
        min_negative_z_cyl1 = intersection_points_cyl1_transformed[min_negative_z_cyl1_idx].reshape(1, 3)
        #print(min_negative_z_cyl1)
        min_distance_neg_1 = min_negative_z_cyl1[0, 2]

    # Step 2: Find the index and points with non-negative and non-positive Z-values for Cylinder 2
    if np.any(intersection_points_cyl2_transformed[:, 2] >= 0):
        # Filter non-negative Z-values (including zero)
        non_negative_z_values_cyl2 = intersection_points_cyl2_transformed[:, 2][intersection_points_cyl2_transformed[:, 2] >= 0]
        min_positive_z_cyl2_idx_filtered = np.argmin(np.abs(non_negative_z_values_cyl2))
        # Map back to original index
        min_positive_z_cyl2_idx = np.where(intersection_points_cyl2_transformed[:, 2] >= 0)[0][min_positive_z_cyl2_idx_filtered]
        min_positive_z_cyl2 = intersection_points_cyl2_transformed[min_positive_z_cyl2_idx].reshape(1, 3)
        #print(min_positive_z_cyl2)
        min_distance_pos_2 = min_positive_z_cyl2[0, 2]

    if np.any(intersection_points_cyl2_transformed[:, 2] <= 0):
        # Filter non-positive Z-values (including zero)
        non_positive_z_values_cyl2 = intersection_points_cyl2_transformed[:, 2][intersection_points_cyl2_transformed[:, 2] <= 0]
        min_negative_z_cyl2_idx_filtered = np.argmin(np.abs(non_positive_z_values_cyl2))
        # Map back to original index
        min_negative_z_cyl2_idx = np.where(intersection_points_cyl2_transformed[:, 2] <= 0)[0][min_negative_z_cyl2_idx_filtered]
        min_negative_z_cyl2 = intersection_points_cyl2_transformed[min_negative_z_cyl2_idx].reshape(1, 3)
        #print(min_negative_z_cyl2)
        min_distance_neg_2 = min_negative_z_cyl2[0, 2]


    # Step 3: Transform the points back to the inertial frame, if they were found
    min_positive_z_cyl1_inertial = apply_transform(ti1, min_positive_z_cyl1) if min_positive_z_cyl1 is not None else None
    #print(min_negative_z_cyl1.shape)
    #print(min_negative_z_cyl1)
    min_negative_z_cyl1_inertial = apply_transform(ti1, min_negative_z_cyl1) if min_negative_z_cyl1 is not None else None
    min_positive_z_cyl2_inertial = apply_transform(ti2, min_positive_z_cyl2) if min_positive_z_cyl2 is not None else None
    min_negative_z_cyl2_inertial = apply_transform(ti2, min_negative_z_cyl2) if min_negative_z_cyl2 is not None else None

    return (
        intersection_points_cyl1,
        min_distance_pos_1,
        min_distance_neg_1,
        min_distance_pos_2,
        min_distance_neg_2,
        min_positive_z_cyl1_inertial,
        min_negative_z_cyl1_inertial,
        min_positive_z_cyl2_inertial,
        min_negative_z_cyl2_inertial
    )

def find_intersection_points(cylinder1_kdtree, cylinder1_points, cylinder2_points, ti1, ti2, t1i, t2i, threshold=0.1):
    """
    Finds intersection points between two cylinders, transforms them back, and returns the closest Z points.
    
    Args:
        cylinder1_kdtree: KDTree for Cylinder 1.
        cylinder1_points: Points of Cylinder 1 in its local frame.
        cylinder2_points: Points of Cylinder 2 in its local frame.
        ti1_inv: Inverse transformation matrix for Cylinder 1.
        ti2_inv: Inverse transformation matrix for Cylinder 2.
        threshold: Distance threshold to consider points intersecting.
    
    Returns:
        - Transformed intersection points back into the inertial frame.
        - Closest point with minimum +Z and -Z for both cylinders.
        - Z values and coordinates in the inertial frame.
    """

    # Step 1: Query KDTree for intersections within a threshold
    distances, indices = cylinder1_kdtree.query(cylinder2_points, distance_upper_bound=threshold)
    valid_mask = distances < threshold
    intersection_points_cyl2 = cylinder2_points[valid_mask]

    # Step 2: Transform intersection points back to Cylinder 1 and 2 local coordinates
    intersection_points_cyl1 = cylinder1_points[indices[valid_mask]]
    if len(intersection_points_cyl1) == 0:
        return [], None, None, None, None, None, None, None, None

    # Transform using inverse matrices to local cylinder coordinates
    intersection_points_cyl1_transformed = apply_transform(t1i,intersection_points_cyl1)
    intersection_points_cyl2_transformed = apply_transform(t2i,intersection_points_cyl2)
    # Step 3: Find min +Z and -Z values and corresponding points
    #print(intersection_points_cyl1)
    #print(intersection_points_cyl1_transformed)

    # Initialize the result variables in case no valid points are found
    min_cyl1 = max_cyl1 = None
    min_cyl1_idx = max_cyl1_idx = None
    min_cyl2 = max_cyl2 = None
    min_cyl2_idx = max_cyl2_idx = None
    min_distance_cyl_1 = max_distance_cyl_1 = min_distance_cyl_2 = max_distance_cyl_2 = None


    if np.any(intersection_points_cyl1_transformed):
        min_cyl1_idx = np.argmin(intersection_points_cyl1_transformed[:, 2])
        min_cyl1 = intersection_points_cyl1_transformed[min_cyl1_idx].reshape(1, 3)
        min_distance_cyl_1 = min_cyl1[0, 2]

        max_cyl1_idx = np.argmax(intersection_points_cyl1_transformed[:, 2])
        max_cyl1 = intersection_points_cyl1_transformed[max_cyl1_idx].reshape(1, 3)
        max_distance_cyl_1 = max_cyl1[0, 2]


    if np.any(intersection_points_cyl2_transformed):
        min_cyl2_idx = np.argmin(intersection_points_cyl2_transformed[:, 2])
        min_cyl2 = intersection_points_cyl2_transformed[min_cyl2_idx].reshape(1, 3)
        min_distance_cyl_2 = min_cyl2[0, 2]

        max_cyl2_idx = np.argmax(intersection_points_cyl2_transformed[:, 2])
        max_cyl2 = intersection_points_cyl2_transformed[max_cyl2_idx].reshape(1, 3)
        max_distance_cyl_2 = max_cyl2[0, 2]




    # Step 3: Transform the points back to the inertial frame, if they were found
    min_cyl1_inertial = apply_transform(ti1, min_cyl1) if min_cyl1 is not None else None
    #print(min_negative_z_cyl1.shape)
    #print(min_negative_z_cyl1)
    max_cyl1_inertial = apply_transform(ti1, max_cyl1) if max_cyl1 is not None else None
    min_cyl2_inertial = apply_transform(ti2, min_cyl2) if min_cyl2 is not None else None
    max_cyl2_inertial = apply_transform(ti2, max_cyl2) if max_cyl2 is not None else None

    return (
        intersection_points_cyl1,
        min_distance_cyl_1,
        max_distance_cyl_1,
        min_distance_cyl_2,
        max_distance_cyl_2,
        min_cyl1_inertial,
        max_cyl1_inertial,
        min_cyl2_inertial,
        max_cyl2_inertial
    )

def downsample_grid(grid, factor):
    return grid[::factor, ::factor]


# Function to plot the cylinders, intersection points, and closest points
def plot_cylinders(cylinder1_surface, cylinder2_surface, intersection_points_cyl1, min_distance_cyl_1, max_distance_cyl_1,
                    min_distance_cyl_2, max_distance_cyl_2, 
                    min_cyl1_inertial, max_cyl1_inertial, min_cyl2_inertial, max_cyl2_inertial,
                    center1, center2, a1, a2):
    fig = go.Figure()

    # Add Cylinder 1 as a surface
    fig.add_trace(go.Surface(
        x=cylinder1_surface[0], y=cylinder1_surface[1], z=cylinder1_surface[2],
        colorscale='Blues',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 1 (a1={a1:.2f}, a2={a2:.2f})'
    ))

    # Add Cylinder 2 as a surface
    fig.add_trace(go.Surface(
        x=cylinder2_surface[0], y=cylinder2_surface[1], z=cylinder2_surface[2],
        colorscale='Reds',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 2 (a1={a1:.2f}, a2={a2:.2f})'
    ))

    # Add vector from origin to Cylinder 1 center
    fig.add_trace(go.Scatter3d(
        x=[0, 271.5 + 124], y=[0, 0], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Arm to Measure'
    ))
    fig.add_trace(go.Scatter3d(
        x=[271.5 + 124, center1[0,0]], y=[0, center1[0,1]], z=[0, center1[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Measure Prop Plane'
    ))

    # Add vector from origin to Cylinder 2 center
    fig.add_trace(go.Scatter3d(
        x=[0, (271.5 + 124) * np.cos(np.pi / 3)], y=[0, (271.5 + 124) * np.sin(np.pi / 3)], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Arm to Blower Left'
    ))
    fig.add_trace(go.Scatter3d(
        x=[(271.5 + 124) * np.cos(np.pi / 3), center2[0,0]], y=[(271.5 + 124) * np.sin(np.pi / 3), center2[0,1] ], z=[0, center2[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Blower Left Prop Plane'
    ))

    # Add Intersection Points
    if len(intersection_points_cyl1) > 0:
        fig.add_trace(go.Scatter3d(
            x=intersection_points_cyl1[:, 0], y=intersection_points_cyl1[:, 1], z=intersection_points_cyl1[:, 2],
            mode='markers', marker=dict(size=4, color='green', opacity=0.8), name=f'Intersections'
        ))


    # Add Min Positive and Negative Z Points for Cylinder 1
    if min_cyl1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl1_inertial[0,0]], y=[min_cyl1_inertial[0,1]], z=[min_cyl1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder Measure, Distance={min_distance_cyl_1:.2f}'
        ))
    if max_cyl1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl1_inertial[0,0]], y=[max_cyl1_inertial[0,1]], z=[max_cyl1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder Measure, Distance={max_distance_cyl_1:.2f}'
        ))

    if min_cyl2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl2_inertial[0,0]], y=[min_cyl2_inertial[0,1]], z=[min_cyl2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder Left, Distance={min_distance_cyl_2:.2f}'
        ))
    if max_cyl2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl2_inertial[0,0]], y=[max_cyl2_inertial[0,1]], z=[max_cyl2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder Left, Distance={max_distance_cyl_2:.2f}'
        ))

    # Customize layout
    fig.update_layout(
        title=f"Cylinder Intersection and Range of Intersection (Angle Measure={np.degrees(a1):.1f}°, Angle Left={np.degrees(a2):.1f}°)",
        scene=dict(
            aspectmode='cube',
            xaxis=dict(title="X-axis", range=[-700, 700]),
            yaxis=dict(title="Y-axis", range=[-700, 700]),
            zaxis=dict(title="Z-axis", range=[-700, 700]),
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        showlegend=True
    )

    # Show plot
    fig.show()


# Define parameters
threshold = 5
radius_prop = 304.8/2
armlength = 271.5 + 124
z_offset = -52.7

# Normalised with prop radius
z_sections = [0.5, 0, -0.9, -1.6, -4.5]
z_sections = [1, 0, -1]
z_sections = sorted(z_sections, reverse=False)

resolution_theta = int(np.round(2*np.pi*radius_prop/threshold/1.4))
resolution_z = int(np.round(2*radius_prop*(max(z_sections) - min(z_sections))/threshold/1.4))
#print(resolution_theta)
#print(resolution_z)


data = []

# Define angles to iterate over
A1 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
#A1 = np.linspace(0, 2 * np.pi, 36, endpoint=False)
#A2 = np.linspace(0, 2 * np.pi, 15, endpoint=False)
#A1 = [np.radians(0)]

angList = list(itertools.product(A1, A2))
angList = [(np.radians(240), np.radians(0)), (np.radians(300), np.radians(180))] # Symmetric combination example
angList = [(np.radians(0), np.radians(-96)), (np.radians(0), np.radians(-110))]
angList = [(np.radians(0), np.radians(270)), (np.radians(0), np.radians(285))]
angList = [(np.radians(210), np.radians(330))] # also possible interaction
angList = [(2.3, -1.2)] # audibly changing interference noise
angList = [(np.radians(160), np.radians(-98.1)), (np.radians(20), np.radians(-106.3))] #max positive and negative influence
angList = [(np.radians(0), np.radians(-90))]
angList = [(np.round(a[0], 8), np.round(a[1], 8)) for a in angList]

# Initialize variables to track the previous state of a1 and cached data
previous_a1 = None
cylinder1_points = None
cylinder1_kdtree = None
cylinder1_surface = None

# Iterate through each combination of angles
for a1, a2 in angList:
    # Check if a1 has changed
    if a1 != previous_a1:
        # Update cylinder1 and its associated data only when a1 changes
        TI1, _ = create_transformation_matrix(a1, 0)  # Only need TI1 for cylinder1
        T1I = invert_transformation(TI1)
        origin_point = np.zeros((1, 3))
        center1 = apply_transform(TI1, origin_point)
        
        # Generate and transform Cylinder 1
        cylinder1_points = generate_slipstream(
            radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
        )
        cylinder1_points = apply_transform(TI1, cylinder1_points)
        
        # Build KDTree for Cylinder 1
        cylinder1_kdtree = KDTree(cylinder1_points)
        
        # Generate surface grids for Cylinder 1 (downsample for plotting)
        cylinder1_surface = generate_surface_grids(cylinder1_points, resolution_theta, resolution_z)
        #cylinder1_surface = tuple(downsample_grid(g, factor=5) for g in cylinder1_surface)

        # Update the cached `previous_a1`
        previous_a1 = a1

    # Process Cylinder 2 for the current a2
    _, TI2 = create_transformation_matrix(a1, a2)
    T2I = invert_transformation(TI2)
    origin_point = np.zeros((1, 3))
    center2 = apply_transform(TI2, origin_point)
    
    # Generate and transform Cylinder 2
    cylinder2_points = generate_slipstream(
        radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
    )
    #print(cylinder2_points.shape)
    cylinder2_points = apply_transform(TI2, cylinder2_points)
    #print(cylinder2_points.shape)
    # Find intersection points using precomputed KDTree for Cylinder 1
    (intersection_points_cyl1,
    min_distance_cyl_1,
    max_distance_cyl_1,
    min_distance_cyl_2,
    max_distance_cyl_2,
    min_cyl1_inertial,
    max_cyl1_inertial,
    min_cyl2_inertial,
    max_cyl2_inertial) = find_intersection_points(
    cylinder1_kdtree, cylinder1_points, cylinder2_points, TI1, TI2, T1I, T2I, threshold)

    # Generate surface grids for Cylinder 2 (downsample for plotting)
    cylinder2_surface = generate_surface_grids(cylinder2_points, resolution_theta, resolution_z)
    #cylinder2_surface = tuple(downsample_grid(g, factor=5) for g in cylinder2_surface)

    # Plot cylinders and intersection points
    plot_cylinders(cylinder1_surface, cylinder2_surface, intersection_points_cyl1, min_distance_cyl_1, max_distance_cyl_1,
                    min_distance_cyl_2, max_distance_cyl_2, 
                    min_cyl1_inertial, max_cyl1_inertial, min_cyl2_inertial, max_cyl2_inertial,
                    center1, center2, a1, a2)

    # Store results in the dataframe
    data.append([np.degrees(a1), np.degrees(a2), min_distance_cyl_1, max_distance_cyl_1,
    min_distance_cyl_2, max_distance_cyl_2])

# Create the DataFrame from collected data
#df = pd.DataFrame(data, columns=['a1', 'a2', 'min_distance_pos_1', 'min_distance_neg_1',
#    'min_distance_pos_2', 'min_distance_neg_2'])
#print(df.shape)
#df.replace([np.inf, -np.inf], np.nan, inplace=True)
#df = df.dropna()
#print(df.shape)
#print(df)
df_test = pd.DataFrame(data)
print(df_test)
print(df_test.round())
#df_cleaned = df.dropna(subset='min_distance_neg_2')


In [ ]:
import numpy as np
import plotly.graph_objects as go
import itertools
import pandas as pd
from scipy.spatial import KDTree

# Function to generate a cylinder with variable radius

def generate_slipstream(rad, z_sections, radius_funcs, resolution_theta=50, resolution_z=50):
    z_min, z_max = min(z_sections), max(z_sections)
    z_steps = np.linspace(z_min, z_max, resolution_z)
    theta = np.linspace(0, 2 * np.pi, resolution_theta)

    radius = np.zeros_like(z_steps)
    for i in range(len(z_sections) - 1):
        mask = (z_steps >= z_sections[i]) & (z_steps <= z_sections[i + 1])
        radius[mask] = rad * radius_funcs[i](z_steps[mask])

    # Avoid creating full grids
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    x = radius[:, None] * cos_theta
    y = radius[:, None] * sin_theta
    z = (z_steps * rad * 2)[:, None] + np.zeros_like(theta)

    # Combine points efficiently
    points = np.column_stack((x.ravel(), y.ravel(), z.ravel()))
    return points


# Define radius functions
#def radius_above(z): return 1 + z *0.25
#def radius_contract(z): return 1 + 0*z
#def radius_nf(z): return 1 - (z+0.9)*0.4
#def radius_ff(z): return 1.3 - (z+1.6)*0.3
#radius_funcs = [radius_ff, radius_nf, radius_contract, radius_above]

def radius_above(z): return 1 +z*0
def radius_contract(z): return 1 + z*0
radius_funcs = [radius_above, radius_contract]

# Function to generate surface grids for plotting
def generate_surface_grids(points, resolution_theta, resolution_z):
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    x_grid = x.reshape(resolution_z, resolution_theta)
    y_grid = y.reshape(resolution_z, resolution_theta)
    z_grid = z.reshape(resolution_z, resolution_theta)
    return x_grid, y_grid, z_grid


# Function to create rotation matrices for given angles
def create_transformation_matrix(a1, a2):
    TI1_1 = np.array([
        [1, 0, 0, 0],
        [0, np.cos(a1), -np.sin(a1), 0],
        [0, np.sin(a1), np.cos(a1), 0],
        [0, 0, 0, 1]
    ])
    T1_2 = np.array([
        [1, 0, 0, armlength],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])

    TI2_1 = np.array([
        [np.cos(np.pi / 3), -np.sin(np.pi / 3), 0, 0],
        [np.sin(np.pi / 3), np.cos(np.pi / 3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    T2_2 = np.array([
        [1, 0, 0, armlength],
        [0, np.cos(a2), -np.sin(a2), 0],
        [0, np.sin(a2), np.cos(a2), 0],
        [0, 0, 0, 1]
    ])
    TI2_3 = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z_offset],
        [0, 0, 0, 1]
    ])
    TI1 = np.matmul(TI1_1,T1_2)
    TI2 = np.matmul(np.matmul(TI2_1, T2_2), TI2_3)
    return TI1, TI2

def invert_transformation(matrix):
    """
    Computes the inverse of a 4x4 homogeneous transformation matrix.
    
    Args:
        T (np.ndarray): A 4x4 homogeneous transformation matrix.
    
    Returns:
        np.ndarray: The inverse of the input transformation matrix.
    """
    # Extract rotation matrix (3x3) and translation vector (3x1)
    R = matrix[:3, :3]
    t = matrix[:3, 3]
    
    # Inverse of rotation is just the transpose of R
    R_inv = R.T
    
    # Inverse of translation is -R_inv * t
    t_inv = -np.dot(R_inv, t)
    
    # Construct the inverse transformation matrix
    matrix_inv = np.eye(4)
    matrix_inv[:3, :3] = R_inv
    matrix_inv[:3, 3] = t_inv
    
    return matrix_inv


# Transformation matrix
def apply_transform(matrix, points):
    num_points = points.shape[0]
    homogeneous_points = np.c_[points, np.ones(num_points)]  # Inline addition of the homogeneous coordinate
    return np.dot(homogeneous_points, matrix.T)[:, :3]


#def find_intersection_points(cylinder1_kdtree, cylinder1_points, cylinder2_points, ti1, ti2, t1i, t2i, threshold=0.1):
    """
    Finds intersection points between two cylinders, transforms them back, and returns the closest Z points.
    
    Args:
        cylinder1_kdtree: KDTree for Cylinder 1.
        cylinder1_points: Points of Cylinder 1 in its local frame.
        cylinder2_points: Points of Cylinder 2 in its local frame.
        ti1_inv: Inverse transformation matrix for Cylinder 1.
        ti2_inv: Inverse transformation matrix for Cylinder 2.
        threshold: Distance threshold to consider points intersecting.
    
    Returns:
        - Transformed intersection points back into the inertial frame.
        - Closest point with minimum +Z and -Z for both cylinders.
        - Z values and coordinates in the inertial frame.
    """

    # Step 1: Query KDTree for intersections within a threshold
    distances, indices = cylinder1_kdtree.query(cylinder2_points, distance_upper_bound=threshold)
    valid_mask = distances < threshold
    intersection_points_cyl2 = cylinder2_points[valid_mask]

    # Step 2: Transform intersection points back to Cylinder 1 and 2 local coordinates
    intersection_points_cyl1 = cylinder1_points[indices[valid_mask]]
    if len(intersection_points_cyl1) == 0:
        return [], None, None, None, None, None, None, None, None

    # Transform using inverse matrices to local cylinder coordinates
    intersection_points_cyl1_transformed = apply_transform(t1i,intersection_points_cyl1)
    intersection_points_cyl2_transformed = apply_transform(t2i,intersection_points_cyl2)
    # Step 3: Find min +Z and -Z values and corresponding points
    #print(intersection_points_cyl1)
    #print(intersection_points_cyl1_transformed)

    # Initialize the result variables in case no valid points are found
    min_positive_z_cyl1 = min_negative_z_cyl1 = None
    min_positive_z_cyl1_idx = min_negative_z_cyl1_idx = None
    min_positive_z_cyl2 = min_negative_z_cyl2 = None
    min_positive_z_cyl2_idx = min_negative_z_cyl2_idx = None
    min_distance_pos_1 = min_distance_neg_1 = min_distance_pos_2 = min_distance_neg_2 = None

    # Step 1: Find the index and points with non-negative and non-positive Z-values for Cylinder 1
    if np.any(intersection_points_cyl1_transformed[:, 2] >= 0):
        # Filter non-negative Z-values (including zero)
        non_negative_z_values_cyl1 = intersection_points_cyl1_transformed[:, 2][intersection_points_cyl1_transformed[:, 2] >= 0]
        min_positive_z_cyl1_idx_filtered = np.argmin(np.abs(non_negative_z_values_cyl1))
        # Map back to original index
        min_positive_z_cyl1_idx = np.where(intersection_points_cyl1_transformed[:, 2] >= 0)[0][min_positive_z_cyl1_idx_filtered]
        min_positive_z_cyl1 = intersection_points_cyl1_transformed[min_positive_z_cyl1_idx].reshape(1, 3)
        #print(min_positive_z_cyl1)
        min_distance_pos_1 = min_positive_z_cyl1[0, 2]

    if np.any(intersection_points_cyl1_transformed[:, 2] <= 0):
        # Filter non-positive Z-values (including zero)
        non_positive_z_values_cyl1 = intersection_points_cyl1_transformed[:, 2][intersection_points_cyl1_transformed[:, 2] <= 0]
        min_negative_z_cyl1_idx_filtered = np.argmin(np.abs(non_positive_z_values_cyl1))
        # Map back to original index
        min_negative_z_cyl1_idx = np.where(intersection_points_cyl1_transformed[:, 2] <= 0)[0][min_negative_z_cyl1_idx_filtered]
        min_negative_z_cyl1 = intersection_points_cyl1_transformed[min_negative_z_cyl1_idx].reshape(1, 3)
        #print(min_negative_z_cyl1)
        min_distance_neg_1 = min_negative_z_cyl1[0, 2]

    # Step 2: Find the index and points with non-negative and non-positive Z-values for Cylinder 2
    if np.any(intersection_points_cyl2_transformed[:, 2] >= 0):
        # Filter non-negative Z-values (including zero)
        non_negative_z_values_cyl2 = intersection_points_cyl2_transformed[:, 2][intersection_points_cyl2_transformed[:, 2] >= 0]
        min_positive_z_cyl2_idx_filtered = np.argmin(np.abs(non_negative_z_values_cyl2))
        # Map back to original index
        min_positive_z_cyl2_idx = np.where(intersection_points_cyl2_transformed[:, 2] >= 0)[0][min_positive_z_cyl2_idx_filtered]
        min_positive_z_cyl2 = intersection_points_cyl2_transformed[min_positive_z_cyl2_idx].reshape(1, 3)
        #print(min_positive_z_cyl2)
        min_distance_pos_2 = min_positive_z_cyl2[0, 2]

    if np.any(intersection_points_cyl2_transformed[:, 2] <= 0):
        # Filter non-positive Z-values (including zero)
        non_positive_z_values_cyl2 = intersection_points_cyl2_transformed[:, 2][intersection_points_cyl2_transformed[:, 2] <= 0]
        min_negative_z_cyl2_idx_filtered = np.argmin(np.abs(non_positive_z_values_cyl2))
        # Map back to original index
        min_negative_z_cyl2_idx = np.where(intersection_points_cyl2_transformed[:, 2] <= 0)[0][min_negative_z_cyl2_idx_filtered]
        min_negative_z_cyl2 = intersection_points_cyl2_transformed[min_negative_z_cyl2_idx].reshape(1, 3)
        #print(min_negative_z_cyl2)
        min_distance_neg_2 = min_negative_z_cyl2[0, 2]


    # Step 3: Transform the points back to the inertial frame, if they were found
    min_positive_z_cyl1_inertial = apply_transform(ti1, min_positive_z_cyl1) if min_positive_z_cyl1 is not None else None
    #print(min_negative_z_cyl1.shape)
    #print(min_negative_z_cyl1)
    min_negative_z_cyl1_inertial = apply_transform(ti1, min_negative_z_cyl1) if min_negative_z_cyl1 is not None else None
    min_positive_z_cyl2_inertial = apply_transform(ti2, min_positive_z_cyl2) if min_positive_z_cyl2 is not None else None
    min_negative_z_cyl2_inertial = apply_transform(ti2, min_negative_z_cyl2) if min_negative_z_cyl2 is not None else None

    return (
        intersection_points_cyl1,
        min_distance_pos_1,
        min_distance_neg_1,
        min_distance_pos_2,
        min_distance_neg_2,
        min_positive_z_cyl1_inertial,
        min_negative_z_cyl1_inertial,
        min_positive_z_cyl2_inertial,
        min_negative_z_cyl2_inertial
    )

def find_intersection_points(cylinder1_kdtree, cylinder1_points, cylinder2_points, ti1, ti2, t1i, t2i, threshold=0.1):
    """
    Finds intersection points between two cylinders, transforms them back, and returns the closest Z points.
    
    Args:
        cylinder1_kdtree: KDTree for Cylinder 1.
        cylinder1_points: Points of Cylinder 1 in its local frame.
        cylinder2_points: Points of Cylinder 2 in its local frame.
        ti1_inv: Inverse transformation matrix for Cylinder 1.
        ti2_inv: Inverse transformation matrix for Cylinder 2.
        threshold: Distance threshold to consider points intersecting.
    
    Returns:
        - Transformed intersection points back into the inertial frame.
        - Closest point with minimum +Z and -Z for both cylinders.
        - Z values and coordinates in the inertial frame.
    """

    # Step 1: Query KDTree for intersections within a threshold
    distances, indices = cylinder1_kdtree.query(cylinder2_points, distance_upper_bound=threshold)
    valid_mask = distances < threshold
    intersection_points_cyl2 = cylinder2_points[valid_mask]

    # Step 2: Transform intersection points back to Cylinder 1 and 2 local coordinates
    intersection_points_cyl1 = cylinder1_points[indices[valid_mask]]
    if len(intersection_points_cyl1) == 0:
        return [], None, None, None, None, None, None, None, None

    # Transform using inverse matrices to local cylinder coordinates
    intersection_points_cyl1_transformed = apply_transform(t1i,intersection_points_cyl1)
    intersection_points_cyl2_transformed = apply_transform(t2i,intersection_points_cyl2)
    # Step 3: Find min +Z and -Z values and corresponding points
    #print(intersection_points_cyl1)
    #print(intersection_points_cyl1_transformed)

    # Initialize the result variables in case no valid points are found
    min_cyl1 = max_cyl1 = None
    min_cyl1_idx = max_cyl1_idx = None
    min_cyl2 = max_cyl2 = None
    min_cyl2_idx = max_cyl2_idx = None
    min_distance_cyl_1 = max_distance_cyl_1 = min_distance_cyl_2 = max_distance_cyl_2 = None


    if np.any(intersection_points_cyl1_transformed):
        min_cyl1_idx = np.argmin(intersection_points_cyl1_transformed[:, 2])
        min_cyl1 = intersection_points_cyl1_transformed[min_cyl1_idx].reshape(1, 3)
        min_distance_cyl_1 = min_cyl1[0, 2]

        max_cyl1_idx = np.argmax(intersection_points_cyl1_transformed[:, 2])
        max_cyl1 = intersection_points_cyl1_transformed[max_cyl1_idx].reshape(1, 3)
        max_distance_cyl_1 = max_cyl1[0, 2]


    if np.any(intersection_points_cyl2_transformed):
        min_cyl2_idx = np.argmin(intersection_points_cyl2_transformed[:, 2])
        min_cyl2 = intersection_points_cyl2_transformed[min_cyl2_idx].reshape(1, 3)
        min_distance_cyl_2 = min_cyl2[0, 2]

        max_cyl2_idx = np.argmax(intersection_points_cyl2_transformed[:, 2])
        max_cyl2 = intersection_points_cyl2_transformed[max_cyl2_idx].reshape(1, 3)
        max_distance_cyl_2 = max_cyl2[0, 2]




    # Step 3: Transform the points back to the inertial frame, if they were found
    min_cyl1_inertial = apply_transform(ti1, min_cyl1) if min_cyl1 is not None else None
    #print(min_negative_z_cyl1.shape)
    #print(min_negative_z_cyl1)
    max_cyl1_inertial = apply_transform(ti1, max_cyl1) if max_cyl1 is not None else None
    min_cyl2_inertial = apply_transform(ti2, min_cyl2) if min_cyl2 is not None else None
    max_cyl2_inertial = apply_transform(ti2, max_cyl2) if max_cyl2 is not None else None

    return (
        intersection_points_cyl1,
        min_distance_cyl_1,
        max_distance_cyl_1,
        min_distance_cyl_2,
        max_distance_cyl_2,
        min_cyl1_inertial,
        max_cyl1_inertial,
        min_cyl2_inertial,
        max_cyl2_inertial
    )

def downsample_grid(grid, factor):
    return grid[::factor, ::factor]


# Function to plot the cylinders, intersection points, and closest points
def plot_cylinders(cylinder1_surface, cylinder2_surface, intersection_points_cyl1, min_distance_cyl_1, max_distance_cyl_1,
                    min_distance_cyl_2, max_distance_cyl_2, 
                    min_cyl1_inertial, max_cyl1_inertial, min_cyl2_inertial, max_cyl2_inertial,
                    center1, center2, a1, a2):
    fig = go.Figure()

    # Add Cylinder 1 as a surface
    fig.add_trace(go.Surface(
        x=cylinder1_surface[0], y=cylinder1_surface[1], z=cylinder1_surface[2],
        colorscale='Blues',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 1 (a1={a1:.2f}, a2={a2:.2f})'
    ))

    # Add Cylinder 2 as a surface
    fig.add_trace(go.Surface(
        x=cylinder2_surface[0], y=cylinder2_surface[1], z=cylinder2_surface[2],
        colorscale='Reds',
        opacity=0.7,
        showscale=False,
        name=f'Cylinder 2 (a1={a1:.2f}, a2={a2:.2f})'
    ))

    # Add vector from origin to Cylinder 1 center
    fig.add_trace(go.Scatter3d(
        x=[0, 271.5 + 124], y=[0, 0], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Arm to Measure'
    ))
    fig.add_trace(go.Scatter3d(
        x=[271.5 + 124, center1[0,0]], y=[0, center1[0,1]], z=[0, center1[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='blue'),
        line=dict(color='blue', width=5),
        name='Measure Prop Plane'
    ))

    # Add vector from origin to Cylinder 2 center
    fig.add_trace(go.Scatter3d(
        x=[0, (271.5 + 124) * np.cos(np.pi / 3)], y=[0, (271.5 + 124) * np.sin(np.pi / 3)], z=[0, 0],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Arm to Blower Left'
    ))
    fig.add_trace(go.Scatter3d(
        x=[(271.5 + 124) * np.cos(np.pi / 3), center2[0,0]], y=[(271.5 + 124) * np.sin(np.pi / 3), center2[0,1] ], z=[0, center2[0,2]],
        mode='lines+markers',
        marker=dict(size=5, color='red'),
        line=dict(color='red', width=5),
        name='Blower Left Prop Plane'
    ))

    # Add Intersection Points
    if len(intersection_points_cyl1) > 0:
        fig.add_trace(go.Scatter3d(
            x=intersection_points_cyl1[:, 0], y=intersection_points_cyl1[:, 1], z=intersection_points_cyl1[:, 2],
            mode='markers', marker=dict(size=4, color='green', opacity=0.8), name=f'Intersections'
        ))


    # Add Min Positive and Negative Z Points for Cylinder 1
    if min_cyl1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl1_inertial[0,0]], y=[min_cyl1_inertial[0,1]], z=[min_cyl1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder Measure, Distance={min_distance_cyl_1:.2f}'
        ))
    if max_cyl1_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl1_inertial[0,0]], y=[max_cyl1_inertial[0,1]], z=[max_cyl1_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder Measure, Distance={max_distance_cyl_1:.2f}'
        ))

    if min_cyl2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[min_cyl2_inertial[0,0]], y=[min_cyl2_inertial[0,1]], z=[min_cyl2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='cyan'), name=f'Min Cylinder Left, Distance={min_distance_cyl_2:.2f}'
        ))
    if max_cyl2_inertial is not None:
        fig.add_trace(go.Scatter3d(
            x=[max_cyl2_inertial[0,0]], y=[max_cyl2_inertial[0,1]], z=[max_cyl2_inertial[0,2]],
            mode='markers', marker=dict(size=8, color='blue'), name=f'Max Cylinder Left, Distance={max_distance_cyl_2:.2f}'
        ))

    # Customize layout
    fig.update_layout(
        #title=f"Cylinder Intersection and Range of Intersection (Measuring Angle={np.degrees(a1):.1f}°, Blowing Angle={np.degrees(a2):.1f}°)",
        #title=f"Visualisation of Intersections (Measuring Angle={np.degrees(a1):.1f}°, Blowing Angle={np.degrees(a2):.1f}°)",
        title=f"Visualisation of Intersections",
        scene=dict(
            aspectmode='cube',
            xaxis=dict(title="X-axis", range=[-2000, 2000]),
            yaxis=dict(title="Y-axis", range=[-2000, 2000]),
            zaxis=dict(title="Z-axis", range=[-2000, 2000]),
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        showlegend=False,
        width=800,  # Set the width of the plot
        height=600  # Set the height of the plot
    )

    # Show plot
    fig.show()


# Define parameters
threshold = 5
radius_prop = 304.8/2
armlength = 271.5 + 124
z_offset = -52.7

# Normalised with prop radius
z_sections = [0.5, 0, -0.9, -1.6, -4.5]
z_sections = [1, 0, -1]
z_sections = sorted(z_sections, reverse=False)

resolution_theta = int(np.round(2*np.pi*radius_prop/threshold/1.4))
resolution_z = int(np.round(2*radius_prop*(max(z_sections) - min(z_sections))/threshold/1.4))
#print(resolution_theta)
#print(resolution_z)


data = []

# Define angles to iterate over
A1 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
A2 = np.linspace(0, 2 * np.pi, 4, endpoint=False)
#A1 = np.linspace(0, 2 * np.pi, 36, endpoint=False)
#A2 = np.linspace(0, 2 * np.pi, 15, endpoint=False)
#A1 = [np.radians(0)]

angList = list(itertools.product(A1, A2))
angList = [(np.radians(240), np.radians(0)), (np.radians(300), np.radians(180))] # Symmetric combination example
angList = [(np.radians(0), np.radians(-96)), (np.radians(0), np.radians(-110))]
angList = [(np.radians(0), np.radians(270)), (np.radians(0), np.radians(285))]
angList = [(np.radians(210), np.radians(330))] # also possible interaction
angList = [(2.3, -1.2)] # audibly changing interference noise
angList = [(np.radians(160), np.radians(-98.1)), (np.radians(20), np.radians(-106.3))] #max positive and negative influence
angList = [(np.radians(180), np.radians(-90))]
angList = [(np.round(a[0], 8), np.round(a[1], 8)) for a in angList]

# Initialize variables to track the previous state of a1 and cached data
previous_a1 = None
cylinder1_points = None
cylinder1_kdtree = None
cylinder1_surface = None

# Iterate through each combination of angles
for a1, a2 in angList:
    # Check if a1 has changed
    if a1 != previous_a1:
        # Update cylinder1 and its associated data only when a1 changes
        TI1, _ = create_transformation_matrix(a1, 0)  # Only need TI1 for cylinder1
        T1I = invert_transformation(TI1)
        origin_point = np.zeros((1, 3))
        center1 = apply_transform(TI1, origin_point)
        
        # Generate and transform Cylinder 1
        cylinder1_points = generate_slipstream(
            radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
        )
        cylinder1_points = apply_transform(TI1, cylinder1_points)
        
        # Build KDTree for Cylinder 1
        cylinder1_kdtree = KDTree(cylinder1_points)
        
        # Generate surface grids for Cylinder 1 (downsample for plotting)
        cylinder1_surface = generate_surface_grids(cylinder1_points, resolution_theta, resolution_z)
        #cylinder1_surface = tuple(downsample_grid(g, factor=5) for g in cylinder1_surface)

        # Update the cached `previous_a1`
        previous_a1 = a1

    # Process Cylinder 2 for the current a2
    _, TI2 = create_transformation_matrix(a1, a2)
    T2I = invert_transformation(TI2)
    origin_point = np.zeros((1, 3))
    center2 = apply_transform(TI2, origin_point)
    
    # Generate and transform Cylinder 2
    cylinder2_points = generate_slipstream(
        radius_prop, z_sections, radius_funcs, resolution_theta=resolution_theta, resolution_z=resolution_z
    )
    #print(cylinder2_points.shape)
    cylinder2_points = apply_transform(TI2, cylinder2_points)
    #print(cylinder2_points.shape)
    # Find intersection points using precomputed KDTree for Cylinder 1
    (intersection_points_cyl1,
    min_distance_cyl_1,
    max_distance_cyl_1,
    min_distance_cyl_2,
    max_distance_cyl_2,
    min_cyl1_inertial,
    max_cyl1_inertial,
    min_cyl2_inertial,
    max_cyl2_inertial) = find_intersection_points(
    cylinder1_kdtree, cylinder1_points, cylinder2_points, TI1, TI2, T1I, T2I, threshold)

    # Generate surface grids for Cylinder 2 (downsample for plotting)
    cylinder2_surface = generate_surface_grids(cylinder2_points, resolution_theta, resolution_z)
    #cylinder2_surface = tuple(downsample_grid(g, factor=5) for g in cylinder2_surface)

    # Plot cylinders and intersection points
    plot_cylinders(cylinder1_surface, cylinder2_surface, intersection_points_cyl1, min_distance_cyl_1, max_distance_cyl_1,
                    min_distance_cyl_2, max_distance_cyl_2, 
                    min_cyl1_inertial, max_cyl1_inertial, min_cyl2_inertial, max_cyl2_inertial,
                    center1, center2, a1, a2)

    # Store results in the dataframe
    data.append([np.degrees(a1), np.degrees(a2), min_distance_cyl_1, max_distance_cyl_1,
    min_distance_cyl_2, max_distance_cyl_2])

# Create the DataFrame from collected data
#df = pd.DataFrame(data, columns=['a1', 'a2', 'min_distance_pos_1', 'min_distance_neg_1',
#    'min_distance_pos_2', 'min_distance_neg_2'])
#print(df.shape)
#df.replace([np.inf, -np.inf], np.nan, inplace=True)
#df = df.dropna()
#print(df.shape)
#print(df)
df_test = pd.DataFrame(data)
print(df_test)
print(df_test.round())
#df_cleaned = df.dropna(subset='min_distance_neg_2')


In [ ]:
def calculate_wrapped_range(group):
    sorted_angles = np.sort(group['a2'].values)
    extended_angles = np.concatenate((sorted_angles, sorted_angles + 360))
    
    # Find the largest gap between consecutive angles
    gaps = np.diff(extended_angles)
    max_gap_idx = np.argmax(gaps)
    
    # Calculate wrapped range
    start_angle = extended_angles[max_gap_idx + 1] % 360
    end_angle = extended_angles[max_gap_idx] % 360
    
    #if start_angle < end_angle:
    a2_min, a2_max = start_angle, end_angle
    #else:
    #    a2_min, a2_max = end_angle, start_angle

    return pd.Series({'a2_min': a2_min, 'a2_max': a2_max})

# Group by 'a1' and apply the function
df_cleaned_copy = df_cleaned.copy()
angular_ranges_df = df_cleaned_copy.groupby('a1').apply(calculate_wrapped_range).reset_index()
df_string = angular_ranges_df.to_string()
print(df_string)


In [53]:
# Save the DataFrame as a CSV file with semicolon delimiters
csv_path = "./impingement_right_range.csv"  # File will be saved in the same folder as the notebook
angular_ranges_df.round(1).to_csv(csv_path, sep=';', index=False, header=False)

In [ ]:
import matplotlib.pyplot as plt

column_names = ['a1', 'a2_min', 'a2_max']
angular_ranges_df_1_2 = pd.read_csv('./angular_ranges_sym_1_2.csv', sep=';', names=column_names, header=None)
angular_ranges_df_1_2 = angular_ranges_df
# Read the CSV and assign column names


# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df_1_2 = angular_ranges_df_1_2.copy()

wrapped_df_1_2['a2_max'] = angular_ranges_df_1_2.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)

# Create the plot
plt.figure(figsize=(12, 8))

## Calculate the area between the curves
#area_a2_min = np.trapz(wrapped_df['a2_min'], wrapped_df['a1'])
#area_a2_max = np.trapz(wrapped_df['a2_max'], wrapped_df['a1'])

#area_between_curves = area_a2_max - area_a2_min
# Identify large gaps (where the gap exceeds the step size)
step_size = wrapped_df_1_2['a1'].diff().dropna().min()
wrapped_df_1_2['gap'] = wrapped_df_1_2['a1'].diff().fillna(step_size) > step_size

# Assign group numbers to segments
wrapped_df_1_2['group'] = wrapped_df_1_2['gap'].cumsum()

# Create the plot
plt.figure(figsize=(12, 8))


for group, group_data in wrapped_df_1_2.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a2_min'].to_numpy(),
        group_data['a2_max'].to_numpy(),
        alpha=0.5,
        color='skyblue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), color='blue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), color='red')


# Labels and Titles
plt.xlabel("Angle Measure (degrees)")
plt.ylabel("Angle Left (degrees)")
plt.title("Expected Regions of Influence (Direct Impingement)")
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(0, 361, 30))  # Y-axis (A2) from 0 to 360, steps of 30

# Show the plot
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Read the CSV and assign column names
column_names = ['a1', 'a2_min', 'a2_max']
angular_ranges_df_1_2 = pd.read_csv('./impingement_left_range.csv', sep=';', names=column_names, header=None)
#angular_ranges_df_1_2 = angular_ranges_df
#print(angular_ranges_df_1_2.to_string())
# Adjust angles to range from -180 to 180
def wrap_angle(angle):
    return ((angle + 180) % 360) - 180

# Function to shift angles to the desired range
def shift_angles(angle, min_angle = -50, max_angle = 310):
    return np.mod(angle - min_angle, max_angle - min_angle) + min_angle

angular_ranges_df_1_2['a1'] = angular_ranges_df_1_2['a1'].apply(shift_angles)
angular_ranges_df_1_2['a2_min'] = angular_ranges_df_1_2['a2_min'].apply(wrap_angle)
angular_ranges_df_1_2['a2_max'] = angular_ranges_df_1_2['a2_max'].apply(wrap_angle)

#print(angular_ranges_df_1_2.to_string())

# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df_1_2 = angular_ranges_df_1_2.copy()
wrapped_df_1_2['a2_max'] = wrapped_df_1_2.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)
wrapped_df_1_2 = wrapped_df_1_2.sort_values(by='a1')
print(wrapped_df_1_2.to_string())

## Calculate the area between the curves
area_a2_min = np.trapz(wrapped_df_1_2['a2_min'], wrapped_df_1_2['a1'])
area_a2_max = np.trapz(wrapped_df_1_2['a2_max'], wrapped_df_1_2['a1'])
area = area_a2_max-area_a2_min
print(area_a2_min)
print(area_a2_max)
print(area)
# Identify large gaps (where the gap exceeds the step size)
step_size = wrapped_df_1_2['a1'].diff().dropna().min()
wrapped_df_1_2['gap'] = wrapped_df_1_2['a1'].diff().fillna(step_size) > step_size

# Assign group numbers to segments
wrapped_df_1_2['group'] = wrapped_df_1_2['gap'].cumsum()

# Create the plot
plt.figure(figsize=(10, 5))

for group, group_data in wrapped_df_1_2.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a2_min'].to_numpy(),
        group_data['a2_max'].to_numpy(),
        alpha=0.5,
        color='skyblue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), color='blue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), color='red')

# Labels and Titles
plt.xlabel("Measuring Angle")
plt.ylabel("Blowing Angle")
plt.title("Expected Regions of Influence (Direct Impingement)")
plt.grid(True)
plt.xticks(np.arange(-60, 311, 30))  # X-axis (A1) from -180 to 180, steps of 30
plt.yticks(np.arange(-180, 181, 30))  # Y-axis (A2) from -180 to 180, steps of 30

# Show the plot
plt.show()


In [ ]:
#Left and right

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Read the CSV and assign column names
column_names = ['a1', 'a2_min', 'a2_max']
angular_ranges_df_1_2 = pd.read_csv('./impingement_left_range.csv', sep=';', names=column_names, header=None)
#angular_ranges_df_1_2 = angular_ranges_df
#print(angular_ranges_df_1_2.to_string())
# Adjust angles to range from -180 to 180
def wrap_angle(angle):
    return ((angle + 180) % 360) - 180

# Function to shift angles to the desired range
def shift_angles(angle, min_angle = -180, max_angle = 180):
    return np.mod(angle - min_angle, max_angle - min_angle) + min_angle

angular_ranges_df_1_2['a1'] = angular_ranges_df_1_2['a1'].apply(shift_angles)
angular_ranges_df_1_2['a2_min'] = angular_ranges_df_1_2['a2_min'].apply(wrap_angle)
angular_ranges_df_1_2['a2_max'] = angular_ranges_df_1_2['a2_max'].apply(wrap_angle)

#print(angular_ranges_df_1_2.to_string())

# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df_1_2 = angular_ranges_df_1_2.copy()
wrapped_df_1_2['a2_max'] = wrapped_df_1_2.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)
wrapped_df_1_2 = wrapped_df_1_2.sort_values(by='a1')

# Identify large gaps (where the gap exceeds the step size)
step_size = wrapped_df_1_2['a1'].diff().dropna().min()
wrapped_df_1_2['gap'] = wrapped_df_1_2['a1'].diff().fillna(step_size) > step_size

# Assign group numbers to segments
wrapped_df_1_2['group'] = wrapped_df_1_2['gap'].cumsum()


# Read the CSV and assign column names
column_names = ['a1', 'a3_min', 'a3_max']
angular_ranges_df_1_3 = pd.read_csv('./impingement_right_range.csv', sep=';', names=column_names, header=None)

angular_ranges_df_1_3['a1'] = angular_ranges_df_1_3['a1'].apply(shift_angles)
angular_ranges_df_1_3['a3_min'] = angular_ranges_df_1_3['a3_min'].apply(wrap_angle)
angular_ranges_df_1_3['a3_max'] = angular_ranges_df_1_3['a3_max'].apply(wrap_angle)

# Adjust a3_max if it's smaller than a3_min (wrap-around)
wrapped_df_1_3 = angular_ranges_df_1_3.copy()
wrapped_df_1_3['a3_max'] = wrapped_df_1_3.apply(
    lambda row: row['a3_max'] + 360 if row['a3_max'] < row['a3_min'] else row['a3_max'],
    axis=1
)
wrapped_df_1_3 = wrapped_df_1_3.sort_values(by='a1')

# Identify large gaps (where the gap exceeds the step size)
step_size = wrapped_df_1_3['a1'].diff().dropna().min()
wrapped_df_1_3['gap'] = wrapped_df_1_3['a1'].diff().fillna(step_size) > step_size

# Assign group numbers to segments
wrapped_df_1_3['group'] = wrapped_df_1_3['gap'].cumsum()

# Create the plot
plt.figure(figsize=(12, 8))

for group, group_data in wrapped_df_1_2.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a2_min'].to_numpy(),
        group_data['a2_max'].to_numpy(),
        alpha=0.5,
        color='skyblue',
        label='Left Impingement' if group == 0 else ""  # Add label only for the first group
        )
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), color='blue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), color='blue')


for group, group_data in wrapped_df_1_3.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a3_min'].to_numpy(),
        group_data['a3_max'].to_numpy(),
        alpha=0.5,
        color='LightSalmon',
        label='Right Impingement' if group == 0 else ""  # Add label only for the first group
    )
    plt.plot(group_data['a1'].to_numpy(), group_data['a3_min'].to_numpy(), color='red')
    plt.plot(group_data['a1'].to_numpy(), group_data['a3_max'].to_numpy(), color='red')
    
# Labels and Titles
plt.xlabel("Angle Measure (degrees)")
plt.ylabel("Angle Blowers (degrees)")
plt.title("Expected Regions of Influence (Direct Impingement)")
plt.grid(True)
plt.xticks(np.arange(-180, 181, 30))  # X-axis (A1) from -180 to 180, steps of 30
plt.yticks(np.arange(-180, 181, 30))  # Y-axis (A2) from -180 to 180, steps of 30


plt.legend(loc='upper right')
# Show the plot
plt.show()


In [ ]:
# Create the plot
plt.figure(figsize=(12, 8))

# Plot for the left impingement range
for group, group_data in wrapped_df_1_2.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a2_min'].to_numpy(),
        group_data['a2_max'].to_numpy(),
        alpha=0.5,
        color='skyblue',
        label='Left Impingement' if group == 0 else ""  # Add label only for the first group
    )
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), color='blue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), color='blue')

    # Calculate the bounding box for the left impingement region
    x_min, x_max = group_data['a1'].min(), group_data['a1'].max()
    y_min, y_max = group_data['a2_min'].min(), group_data['a2_max'].max()
    plt.gca().add_patch(plt.Rectangle(
        (x_min, y_min),
        x_max - x_min,
        y_max - y_min,
        linewidth=1.5,
        edgecolor='blue',
        facecolor='none',
        linestyle='--',
        label='Left Impingement Box' if group == 0 else ""  # Add label only for the first group
    ))

# Plot for the right impingement range
for group, group_data in wrapped_df_1_3.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a3_min'].to_numpy(),
        group_data['a3_max'].to_numpy(),
        alpha=0.5,
        color='LightSalmon',
        label='Right Impingement' if group == 0 else ""  # Add label only for the first group
    )
    plt.plot(group_data['a1'].to_numpy(), group_data['a3_min'].to_numpy(), color='red')
    plt.plot(group_data['a1'].to_numpy(), group_data['a3_max'].to_numpy(), color='red')

    # Calculate the bounding box for the right impingement region
    x_min, x_max = group_data['a1'].min(), group_data['a1'].max()
    y_min, y_max = group_data['a3_min'].min(), group_data['a3_max'].max()
    plt.gca().add_patch(plt.Rectangle(
        (x_min, y_min),
        x_max - x_min,
        y_max - y_min,
        linewidth=1.5,
        edgecolor='red',
        facecolor='none',
        linestyle='--',
        label='Right Impingement Box' if group == 0 else ""  # Add label only for the first group
    ))

# Labels and Titles
plt.xlabel("Angle Measure (degrees)")
plt.ylabel("Angle Blowers (degrees)")
plt.title("Expected Regions of Influence (Direct Impingement)")
plt.grid(True)
plt.xticks(np.arange(-180, 181, 30))  # X-axis (A1) from -180 to 180, steps of 30
plt.yticks(np.arange(-180, 181, 30))  # Y-axis (A2) from -180 to 180, steps of 30

# Add legend
plt.legend(loc='upper right')

# Show the plot
plt.show()


In [ ]:
# Helper functions to round to the nearest 30-degree interval
def round_down_to_nearest_30(value):
    return 30 * np.floor(value / 30)

def round_up_to_nearest_30(value):
    return 30 * np.ceil(value / 30)

# Calculate global bounds for left impingement (a2_min and a2_max)
left_y_min = round_down_to_nearest_30(wrapped_df_1_2[['a2_min', 'a2_max']].values.min())
left_y_max = round_up_to_nearest_30(wrapped_df_1_2[['a2_min', 'a2_max']].values.max())

# Calculate global bounds for right impingement (a3_min and a3_max)
right_y_min = round_down_to_nearest_30(wrapped_df_1_3[['a3_min', 'a3_max']].values.min())
right_y_max = round_up_to_nearest_30(wrapped_df_1_3[['a3_min', 'a3_max']].values.max())

# Create the plot
plt.figure(figsize=(10, 5))

# Plot for the left impingement range
for group, group_data in wrapped_df_1_2.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a2_min'].to_numpy(),
        group_data['a2_max'].to_numpy(),
        alpha=0.5,
        color='skyblue',
        #label='Left Impingement' if group == 0 else ""  # Add label only for the first group
    )
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), color='blue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), color='blue')

    # Calculate X bounds for each group
    left_x_min = round_down_to_nearest_30(group_data['a1'].min())
    left_x_max = round_up_to_nearest_30(group_data['a1'].max())

    # Draw the rectangle for the left bounding box
    plt.gca().add_patch(plt.Rectangle(
        (left_x_min, left_y_min),  # Bottom-left corner
        left_x_max - left_x_min,  # Width
        left_y_max - left_y_min,  # Height
        linewidth=1.5,
        edgecolor='blue',
        facecolor='none',
        linestyle='--',
        #label='Left Impingement Box' if group == 0 else ""
    ))

# Plot for the right impingement range
for group, group_data in wrapped_df_1_3.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a3_min'].to_numpy(),
        group_data['a3_max'].to_numpy(),
        alpha=0.5,
        color='LightSalmon',
        #label='Right Impingement' if group == 0 else ""  # Add label only for the first group
    )
    plt.plot(group_data['a1'].to_numpy(), group_data['a3_min'].to_numpy(), color='red')
    plt.plot(group_data['a1'].to_numpy(), group_data['a3_max'].to_numpy(), color='red')

    # Calculate X bounds for each group
    right_x_min = round_down_to_nearest_30(group_data['a1'].min())
    right_x_max = round_up_to_nearest_30(group_data['a1'].max())

    # Draw the rectangle for the right bounding box
    plt.gca().add_patch(plt.Rectangle(
        (right_x_min, right_y_min),  # Bottom-left corner
        right_x_max - right_x_min,  # Width
        right_y_max - right_y_min,  # Height
        linewidth=1.5,
        edgecolor='red',
        facecolor='none',
        linestyle='--',
        #label='Right Impingement Box' if group == 0 else ""
    ))

# Labels and Titles
plt.xlabel("Measuring Angle")
plt.ylabel("Blowing Angles")
plt.title("Expected Regions of Influence for a 3 Prop System")
plt.grid(True)
plt.xticks(np.arange(-180, 181, 30))  # X-axis (A1) from -180 to 180, steps of 30
plt.yticks(np.arange(-180, 181, 30))  # Y-axis (A2) from -180 to 180, steps of 30

# Add legend
plt.legend(loc='upper right')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

column_names = ['a1', 'a2_min', 'a2_max']
angular_ranges_df_1_2 = pd.read_csv('./angular_ranges_sym_1_2.csv', sep=';', names=column_names, header=None)
# Read the CSV and assign column names
angular_ranges_df= pd.read_csv('./angular_ranges_1_2.csv', sep=';', names=column_names, header=None)


# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df_1_2 = angular_ranges_df_1_2.copy()

wrapped_df_1_2['a2_max'] = angular_ranges_df_1_2.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)

# Create the plot
plt.figure(figsize=(12, 8))

## Calculate the area between the curves
#area_a2_min = np.trapz(wrapped_df['a2_min'], wrapped_df['a1'])
#area_a2_max = np.trapz(wrapped_df['a2_max'], wrapped_df['a1'])

#area_between_curves = area_a2_max - area_a2_min
# Identify large gaps (where the gap exceeds the step size)
step_size = wrapped_df_1_2['a1'].diff().dropna().min()
wrapped_df_1_2['gap'] = wrapped_df_1_2['a1'].diff().fillna(step_size) > step_size

# Assign group numbers to segments
wrapped_df_1_2['group'] = wrapped_df_1_2['gap'].cumsum()

# Create the plot
plt.figure(figsize=(12, 8))


for group, group_data in wrapped_df_1_2.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a2_min'].to_numpy(),
        group_data['a2_max'].to_numpy(),
        alpha=0.5,
        color='skyblue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), color='blue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), color='red')



wrapped_df_1_3 = angular_ranges_df.copy()

# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df_1_3['a2_max'] = angular_ranges_df.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)

wrapped_df_1_3['gap'] = wrapped_df_1_3['a1'].diff().fillna(step_size) > step_size

# Assign group numbers to segments
wrapped_df_1_3['group'] = wrapped_df_1_3['gap'].cumsum()


for group, group_data in wrapped_df_1_3.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a2_min'].to_numpy(),
        group_data['a2_max'].to_numpy(),
        alpha=0.5,
        color='lightgreen')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), 'g--')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), 'm--')

# Labels and Titles
plt.xlabel("A1 (degrees)")
plt.ylabel("A2 Range (degrees)")
plt.title("A2 Range for Different A1 Values")
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(0, 421, 30))  # Y-axis (A2) from 0 to 360, steps of 30

# Show the plot
plt.show()

In [ ]:
angle = 210
if angle > 180:
    angle = angle - 360
rad = np.radians(angle)
print(rad)

In [ ]:
np.radians(-68)

In [ ]:
np.degrees(0.25)

In [ ]:
# Read the CSV and assign column names
column_names = ['a1', 'a2_min', 'a2_max']
angular_ranges_df_1_2 = pd.read_csv('./angular_ranges_sym_1_2.csv', sep=';', names=column_names, header=None)

# Read the CSV and assign column names
column_names = ['a1', 'a3_min', 'a3_max']
angular_ranges_df_1_3 = pd.read_csv('./angular_ranges_sym_1_3.csv', sep=';', names=column_names, header=None)

# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df_1_2 = angular_ranges_df_1_2.copy()

wrapped_df_1_2['a2_max'] = angular_ranges_df_1_2.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)

# Create the plot
plt.figure(figsize=(12, 8))


#area_a2_min = np.trapz(wrapped_df['a2_min'], wrapped_df['a1'])
#area_a2_max = np.trapz(wrapped_df['a2_max'], wrapped_df['a1'])
#
## Calculate the area between the curves
#area_between_curves = area_a2_max - area_a2_min
# Actual data: Fill between the a2_min and a2_max
# Identify large gaps (where the gap exceeds the step size)

step_size = wrapped_df_1_2['a1'].diff().dropna().min()
wrapped_df_1_2['gap'] = wrapped_df_1_2['a1'].diff().fillna(step_size) > step_size

# Assign group numbers to segments
wrapped_df_1_2['group'] = wrapped_df_1_2['gap'].cumsum()

# Create the plot
plt.figure(figsize=(12, 8))


for group, group_data in wrapped_df_1_2.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a2_min'].to_numpy(),
        group_data['a2_max'].to_numpy(),
        alpha=0.5,
        color='skyblue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_min'].to_numpy(), color='blue')
    plt.plot(group_data['a1'].to_numpy(), group_data['a2_max'].to_numpy(), color='red')



wrapped_df_1_3 = angular_ranges_df_1_3.copy()

# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df_1_3['a3_min'] = angular_ranges_df_1_3.apply(
    lambda row: row['a3_min'] - 360 if row['a3_max'] < row['a3_min'] else row['a3_min'],
    axis=1
)

step_size = wrapped_df_1_3['a1'].diff().dropna().min()

wrapped_df_1_3['gap'] = wrapped_df_1_3['a1'].diff().fillna(step_size) > step_size

# Assign group numbers to segments
wrapped_df_1_3['group'] = wrapped_df_1_3['gap'].cumsum()


for group, group_data in wrapped_df_1_3.groupby('group'):
    plt.fill_between(
        group_data['a1'].to_numpy(),
        group_data['a3_min'].to_numpy(),
        group_data['a3_max'].to_numpy(),
        alpha=0.5,
        color='lightgreen')
    plt.plot(group_data['a1'].to_numpy(), group_data['a3_min'].to_numpy(), 'g--')
    plt.plot(group_data['a1'].to_numpy(), group_data['a3_max'].to_numpy(), 'm--')


# Labels and Titles
plt.xlabel("Measure (degrees)")
plt.ylabel("Blower Range (degrees)")
plt.title("Blower Ranges for Different A1 Values (Both Sides)")
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(-60, 421, 30))  # Y-axis (A2) from 0 to 420, steps of 30

# Show the plot
plt.show()

In [ ]:
angular_ranges_180_df = angular_ranges_df.copy()
angular_ranges_180_df['a2_min'] = angular_ranges_180_df['a2_min'].apply(lambda x: x - 360 if x > 180 else x)
angular_ranges_180_df['a2_max'] = angular_ranges_180_df['a2_max'].apply(lambda x: x - 360 if x > 180 else x)

In [ ]:
wrapped_df_180 = angular_ranges_180_df.copy()

## Adjust a2_max if it's smaller than a2_min (wrap-around)
#wrapped_df_180['a2_max'] = angular_ranges_df.apply(
#    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
#    axis=1
#)

# Create the plot
plt.figure(figsize=(12, 8))
plt.fill_between(
    wrapped_df_180['a1'], 
    wrapped_df_180['a2_min'], 
    wrapped_df_180['a2_max'], 
    where=(wrapped_df_180['a2_max'] >= wrapped_df_180['a2_min']),
    color='skyblue', 
    alpha=0.5,
    label='Collision Range',
    interpolate=True
)

# Fill areas where a2_min is greater than a2_max
plt.fill_between(
    wrapped_df_180['a1'], 
    wrapped_df_180['a2_min'], 
    180, 
    where=(wrapped_df_180['a2_min'] > wrapped_df_180['a2_max']), 
    color='skyblue', 
    alpha=0.5,
    interpolate=True
)

plt.fill_between(
    wrapped_df_180['a1'], 
    -180, 
    wrapped_df_180['a2_max'], 
    where=(wrapped_df_180['a2_min'] > wrapped_df_180['a2_max']), 
    color='skyblue', 
    alpha=0.5,
    interpolate=True
)

#print(angular_ranges_df['a1'])
# Plot A2 min and A2 max as lines
plt.plot(wrapped_df_180['a1'].values, wrapped_df_180['a2_min'].values, color='blue', label='A2 Min')
plt.plot(wrapped_df_180['a1'].values, wrapped_df_180['a2_max'].values, color='red', label='A2 Max')


# Labels and Titles
plt.xlabel("A1 (degrees)")
plt.ylabel("A2 Range (degrees)")
plt.title("A2 Range from -180 to 180 for Different A1 Values")
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(-180, 181, 30))  # Y-axis (A2) from 0 to 360, steps of 30

# Show the plot
plt.show()

In [ ]:
df_sorted_sym = df_cleaned.sort_values(['min_distance_pos_1', 'min_distance_neg_1', 'min_distance_pos_2', 'min_distance_neg_2'])
df_string_sym = df_sorted_sym.to_string()
print(df_string_sym)

In [ ]:
from scipy.optimize import curve_fit

# Creating the DataFrame from your provided data
data_1 = {
    'a1': [0.0, 30.0, 60.0, 90.0, 120.0, 150.0, 180.0, 210.0, 240.0, 270.0, 300.0, 330.0, 360.0],
    'a2_min': [210.0, 240.0, 210.0, 150.0, 150.0, 180.0, 210.0, 240.0, 210.0, 180.0, 180.0, 180.0, 210.0],
    'a2_max': [330.0, 0.0, 30.0, 30.0, 330.0, 300.0, 330.0, 0.0, 0.0, 0.0, 330.0, 300.0, 330.0]
}

df_fake = pd.DataFrame(data_1)
df_fake = df_fake.copy()

column_names = ['a1', 'a2_min', 'a2_max']

# Read the CSV and assign column names
df_fake = pd.read_csv('./angular_ranges_1_2.csv', sep=';', names=column_names, header=None)
#print(df_fake.head())


df_fake['a2_max'] = df_fake.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)

# Trigonometric model
def trig_model_min(x, a, b, c, d, e, f, g, h):
    """A simple trigonometric model for fitting."""
    return a * np.cos(b * np.radians(x) + c) + d + e * x + f *np.sin(g * np.radians(x) + h)

def trig_model_max(x, a, b, c, d, e, f):
    """A simple trigonometric model for fitting."""
    return a * np.sin(b * np.radians(x) + c) + d + e * (np.radians(x) // np.pi) + f

# Fitting the model to the data for a2_min and a2_max
#popt_min, _ = curve_fit(trig_model_min, df_fake['a1'], df_fake['a2_min'], p0=[1, 1, 0, 200, 0])
#popt_max, _ = curve_fit(trig_model_max, df_fake['a1'], df_fake['a2_max'], p0=[1, 2, 0, 360, 0, 300])
popt_min, _ = curve_fit(trig_model_min, df_fake['a1'], df_fake['a2_min'])
popt_max, _ = curve_fit(trig_model_max, df_fake['a1'], df_fake['a2_max'])
#print(popt_min)
#print(popt_max)
#popt_max =[ 38.25871357,  2, 200.91393116, 117.16842021,   0.33155818]
a1_values = df['a1'].values
a2_min_pred = trig_model_min(a1_values, *popt_min)
a2_max_pred = trig_model_max(a1_values, *popt_max)

# Store predictions in an array
data_pred = np.column_stack((a1_values, a2_min_pred, a2_max_pred))

# Create a DataFrame from the predicted data
predicted_df = pd.DataFrame(data_pred, columns=['a1', 'a2_min_pred', 'a2_max_pred'])

# Now you can use the fitted model to predict a2_min and a2_max for new a1 values
def predict_a2(a1_value):
    a2_min_pred = trig_model_min(a1_value, *popt_min)
    a2_max_pred = trig_model_max(a1_value, *popt_max)
    return a2_min_pred, a2_max_pred

# Example usage
a1_input = 45.0  # You can change this value
a2_min_output, a2_max_output = predict_a2(a1_input)
#print(f"For a1={a1_input}, predicted a2_min={a2_min_output} and a2_max={a2_max_output}")

In [ ]:
# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df = angular_ranges_df.copy()
wrapped_df['a2_max'] = angular_ranges_df.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)

# Create the plot
plt.figure(figsize=(12, 8))

# Actual data: Fill between the a2_min and a2_max
plt.fill_between(
    wrapped_df['a1'].values,  # Convert to 1D array
    wrapped_df['a2_min'].values,  # Convert to 1D array
    wrapped_df['a2_max'].values,  # Convert to 1D array
    color='skyblue',
    alpha=0.5,
    label='Collision Range (Actual)'
)

# Actual data: Plot A2 min and A2 max as lines
plt.plot(wrapped_df['a1'].values, wrapped_df['a2_min'].values, color='blue', label='A2 Min (Actual)')
plt.plot(wrapped_df['a1'].values, wrapped_df['a2_max'].values, color='red', label='A2 Max (Actual)')


wrapped_predicted_df = predicted_df.copy()
wrapped_predicted_df['a2_max_pred'] = predicted_df.apply(
    lambda row: row['a2_max_pred'] + 360 if row['a2_max_pred'] < row['a2_min_pred'] else row['a2_max_pred'],
    axis=1
)
# Predicted data: Fill between the predicted a2_min and a2_max
plt.fill_between(
    wrapped_predicted_df['a1'].values,
    wrapped_predicted_df['a2_min_pred'].values,
    wrapped_predicted_df['a2_max_pred'].values,
    color='lightgreen',
    alpha=0.5,
    label='Collision Range (Predicted)'
)

# Predicted data: Plot A2 min and A2 max as dashed lines
plt.plot(wrapped_predicted_df['a1'].values, wrapped_predicted_df['a2_min_pred'].values, 'g--', label='A2 Min (Predicted)')
plt.plot(wrapped_predicted_df['a1'].values, wrapped_predicted_df['a2_max_pred'].values, 'm--', label='A2 Max (Predicted)')

# Labels and Titles
plt.xlabel("A1 (degrees)")
plt.ylabel("A2 Range (degrees)")
plt.title("A2 Range for Different A1 Values (Actual vs Predicted)")
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(0, 421, 30))  # Y-axis (A2) from 0 to 420, steps of 30

# Show the plot
plt.show()

In [ ]:
column_names = ['a1', 'a2_min', 'a2_max']

# Read the CSV and assign column names
angular_ranges_df_1_2 = pd.read_csv('./angular_ranges_1_2.csv', sep=';', names=column_names, header=None)

# Read the CSV and assign column names
angular_ranges_df_1_3 = pd.read_csv('./angular_ranges_1_3.csv', sep=';', names=column_names, header=None)

# Adjust a2_max if it's smaller than a2_min (wrap-around)
wrapped_df_1_2 = angular_ranges_df_1_2.copy()
wrapped_df_1_2['a2_max'] = angular_ranges_df_1_2.apply(
    lambda row: row['a2_max'] + 360 if row['a2_max'] < row['a2_min'] else row['a2_max'],
    axis=1
)

# Create the plot
plt.figure(figsize=(12, 8))

# Actual data: Fill between the a2_min and a2_max
plt.fill_between(
    wrapped_df_1_2['a1'].values,  # Convert to 1D array
    wrapped_df_1_2['a2_min'].values,  # Convert to 1D array
    wrapped_df_1_2['a2_max'].values,  # Convert to 1D array
    color='skyblue',
    alpha=0.5,
    label='Collision Range'
)

# Actual data: Plot A2 min and A2 max as lines
plt.plot(wrapped_df_1_2['a1'].values, wrapped_df_1_2['a2_min'].values, color='blue', label='A2 Min')
plt.plot(wrapped_df_1_2['a1'].values, wrapped_df_1_2['a2_max'].values, color='red', label='A2 Max')


wrapped_df_1_3 = angular_ranges_df_1_3.copy()
wrapped_df_1_3['a2_min'] = angular_ranges_df_1_3.apply(
    lambda row: row['a2_min'] - 360 if row['a2_max'] < row['a2_min'] else row['a2_min'],
    axis=1
)
# Predicted data: Fill between the predicted a2_min and a2_max
plt.fill_between(
    wrapped_df_1_3['a1'].values,
    wrapped_df_1_3['a2_min'].values,
    wrapped_df_1_3['a2_max'].values,
    color='lightgreen',
    alpha=0.5,
    label='Collision Range'
)

# Predicted data: Plot A2 min and A2 max as dashed lines
plt.plot(wrapped_df_1_3['a1'].values, wrapped_df_1_3['a2_min'].values, 'g--', label='A3 Min')
plt.plot(wrapped_df_1_3['a1'].values, wrapped_df_1_3['a2_max'].values, 'm--', label='A3 Max')

# Labels and Titles
plt.xlabel("Measure (degrees)")
plt.ylabel("Blower Range (degrees)")
plt.title("Blower Ranges for Different A1 Values (Both Sides)")
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0, 361, 30))  # X-axis (A1) from 0 to 360, steps of 30
plt.yticks(np.arange(-30, 421, 30))  # Y-axis (A2) from 0 to 420, steps of 30

# Show the plot
plt.show()

In [ ]:
def generate_a2_dataframe(df, value_range, resolution):
    data = []

    for _, row in df.iterrows():
        a1 = row['a1']
        
        # Generate a2 range around a2_min
        a2_min_values = np.arange(row['a2_min'] - value_range, row['a2_min'] + value_range + resolution, resolution)
        
        # Generate a2 range around a2_max
        a2_max_values = np.arange(row['a2_max'] - value_range, row['a2_max'] + value_range + resolution, resolution)
        
        # Combine the two ranges
        for a2 in np.concatenate((a2_min_values, a2_max_values)):
            data.append({'a1': a1, 'a2': a2})
    
    # Create the DataFrame
    result_df = pd.DataFrame(data)
    return result_df

value_range = 10 # ±30 degrees
resolution = 5   # Step size of 10 degrees

# Generate the DataFrame
generated_df = generate_a2_dataframe(angular_ranges_df, value_range, resolution)
print(generated_df.to_string())

In [ ]:
# Save the DataFrame as a CSV file with semicolon delimiters
csv_path = "./generated_a2_data.csv"  # File will be saved in the same folder as the notebook
generated_df.to_csv(csv_path, sep=';', index=False, header=False)

In [ ]:
A1 = np.linspace(0, 360, 4, endpoint=False)
A2 = np.linspace(0, 360, 4, endpoint=False)
A1 = [(0)]
TestList = list(itertools.product(A1, A2))
# Create a DataFrame
df_testing = pd.DataFrame(TestList, columns=['A1', 'A2'])
# Update the values: if greater than 180, subtract 360
df_testing['A1'] = df_testing['A1'].apply(lambda x: x - 360 if x > 180 else x)
df_testing['A2'] = df_testing['A2'].apply(lambda x: x - 360 if x > 180 else x)

print(df_testing)

In [ ]:
# Save the DataFrame as a CSV file with semicolon delimiters
csv_path = "./a1_and_a2_influence_test.csv"  # File will be saved in the same folder as the notebook
df_testing.to_csv(csv_path, sep=';', index=False, header=False)

make csv file out of df ( semilcolons as limiters, new line for new line)
1d above 1d below